# Flag QC-failed imaging wells 

In [85]:
# Imports
import os
import warnings
import numpy as np
import polars as pl
import pandas as pd
from tqdm import tqdm
import statsmodels.formula.api as smf
from scipy.stats import ttest_rel

PLATEMAP_DIR = "../../../8.2_updated_snakemake_pipeline/inputs/metadata/platemaps/{batch_id}/platemap"
IMGS_QC_DIR = "../../../8.1_upstream_analysis_runxi/2.raw_img_qc/outputs/1.plate_bg_summary"
CELL_CHANGES = ["localization", "morphology"]

# PROF_DIR = "../../../8.2_updated_snakemake_pipeline/outputs/batch_profiles"
# CLASS_RES_DIR = "../../../8.2_updated_snakemake_pipeline/outputs/classification_results"
# TIFF_IMGS_DIR = "../../../8.1_upstream_analysis_runxi/2.raw_img_qc/inputs/images"

## Batch 7-8

In [102]:
BIO_REP_BATCHES = ["2024_01_23_Batch_7", "2024_02_06_Batch_8"]
COMBINED_BIO_REP_DIR = "2024_02_Batch_7-8"

OUTPUT_DIR = f"../../outputs/{COMBINED_BIO_REP_DIR}"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [103]:
allele_meta_dict = {}
for batch_id in BIO_REP_BATCHES:
    allele_meta_df = pl.DataFrame()
    for platemap in os.listdir(PLATEMAP_DIR.format(batch_id=batch_id)):
        platemap_df = pl.read_csv(os.path.join(PLATEMAP_DIR.format(batch_id=batch_id), platemap), separator="\t", infer_schema_length=100000)
        allele_meta_df = pl.concat([allele_meta_df, 
                                    platemap_df.filter((~pl.col("node_type").is_in(["TC","NC","PC"]))&(pl.col("node_type").is_not_null()))], 
                                    how="diagonal_relaxed").sort("plate_map_name")
    allele_meta_dict[batch_id] = allele_meta_df
    display(allele_meta_df)#.sort()

plate_map_name,well_position,control_type,symbol,gene_allele,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,batch1,batch2,clone_ID,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,vector,notes,plate,quad,c96,r96,r384,c384,r384_letter,gene_symbol,symbol_original
str,str,str,str,str,str,str,str,str,f64,f64,str,str,f64,i64,i64,i64,str,str,str,str,f64,str,i64,i64,i64,i64,i64,i64,str,str,str
"""B7A1R1_P1""","""A01""","""disease_wt""","""ACSF3""","""ACSF3""","""A01""","""B7A1R1_P1""","""B8A1R2_P1""","""disease_wt""",71337.0,null,null,null,1.0,1,7,8,null,null,null,null,527.0,null,1,1,1,1,1,1,"""A""","""ACSF3""","""ACSF3"""
"""B7A1R1_P1""","""C01""","""allele""","""ACSF3""","""ACSF3_Ala197Thr""","""C01""","""B7A1R1_P1""","""B8A1R2_P1""","""allele""",71337.0,3943.0,"""589G>A""","""Ala197Thr""",1.0,1,7,8,null,null,null,null,527.0,null,1,1,1,2,3,1,"""C""","""ACSF3""","""ACSF3"""
"""B7A1R1_P1""","""E01""","""allele""","""ACSF3""","""ACSF3_Thr358Ile""","""E01""","""B7A1R1_P1""","""B8A1R2_P1""","""allele""",71337.0,205771.0,"""1073C>T""","""Thr358Ile""",1.0,1,7,8,null,null,null,null,527.0,null,1,1,1,3,5,1,"""E""","""ACSF3""","""ACSF3"""
"""B7A1R1_P1""","""G01""","""allele""","""ACSF3""","""ACSF3_Met198Arg""","""G01""","""B7A1R1_P1""","""B8A1R2_P1""","""allele""",71337.0,3944.0,"""593T>G""","""Met198Arg""",1.0,1,7,8,null,null,null,null,527.0,null,1,1,1,4,7,1,"""G""","""ACSF3""","""ACSF3"""
"""B7A1R1_P1""","""I01""","""allele""","""ACSF3""","""ACSF3_Glu359Lys""","""I01""","""B7A1R1_P1""","""B8A1R2_P1""","""allele""",71337.0,205770.0,"""1075G>A""","""Glu359Lys""",1.0,1,7,8,null,null,null,null,527.0,null,1,1,1,5,9,1,"""I""","""ACSF3""","""ACSF3"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""B7A1R1_P4""","""L08""","""disease_wt""","""PTPRS""","""PTPRS""","""L08""","""B7A1R1_P4""","""B8A1R2_P4""","""disease_wt""",null,null,null,null,1.0,4,7,8,null,null,null,null,527.0,null,4,4,4,6,12,8,"""L""","""PTPRS""","""PTPRS"""
"""B7A1R1_P4""","""N08""","""allele""","""PTPRS""","""PTPRS_Pro548Leu""","""N08""","""B7A1R1_P4""","""B8A1R2_P4""","""allele""",null,null,null,"""Pro548Leu""",1.0,4,7,8,null,null,null,null,527.0,null,4,4,4,7,14,8,"""N""","""PTPRS""","""PTPRS"""
"""B7A1R1_P4""","""P08""","""disease_wt""","""TEAD2""","""TEAD2""","""P08""","""B7A1R1_P4""","""B8A1R2_P4""","""disease_wt""",null,null,null,null,1.0,4,7,8,null,null,null,null,527.0,null,4,4,4,8,16,8,"""P""","""TEAD2""","""TEAD2"""


plate_map_name,well_position,control_type,symbol,gene_allele,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,batch1,batch2,clone_ID,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,vector,notes,plate,quad,c96,r96,r384,c384,r384_letter,gene_symbol,symbol_original
str,str,str,str,str,str,str,str,str,f64,f64,str,str,f64,i64,i64,i64,str,str,str,str,f64,str,i64,i64,i64,i64,i64,i64,str,str,str
"""B8A1R2_P1""","""A01""","""disease_wt""","""ACSF3""","""ACSF3""","""A01""","""B7A1R1_P1""","""B8A1R2_P1""","""disease_wt""",71337.0,null,null,null,1.0,1,7,8,null,null,null,null,527.0,null,1,1,1,1,1,1,"""A""","""ACSF3""","""ACSF3"""
"""B8A1R2_P1""","""C01""","""allele""","""ACSF3""","""ACSF3_Ala197Thr""","""C01""","""B7A1R1_P1""","""B8A1R2_P1""","""allele""",71337.0,3943.0,"""589G>A""","""Ala197Thr""",1.0,1,7,8,null,null,null,null,527.0,null,1,1,1,2,3,1,"""C""","""ACSF3""","""ACSF3"""
"""B8A1R2_P1""","""E01""","""allele""","""ACSF3""","""ACSF3_Thr358Ile""","""E01""","""B7A1R1_P1""","""B8A1R2_P1""","""allele""",71337.0,205771.0,"""1073C>T""","""Thr358Ile""",1.0,1,7,8,null,null,null,null,527.0,null,1,1,1,3,5,1,"""E""","""ACSF3""","""ACSF3"""
"""B8A1R2_P1""","""G01""","""allele""","""ACSF3""","""ACSF3_Met198Arg""","""G01""","""B7A1R1_P1""","""B8A1R2_P1""","""allele""",71337.0,3944.0,"""593T>G""","""Met198Arg""",1.0,1,7,8,null,null,null,null,527.0,null,1,1,1,4,7,1,"""G""","""ACSF3""","""ACSF3"""
"""B8A1R2_P1""","""I01""","""allele""","""ACSF3""","""ACSF3_Glu359Lys""","""I01""","""B7A1R1_P1""","""B8A1R2_P1""","""allele""",71337.0,205770.0,"""1075G>A""","""Glu359Lys""",1.0,1,7,8,null,null,null,null,527.0,null,1,1,1,5,9,1,"""I""","""ACSF3""","""ACSF3"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""B8A1R2_P4""","""L08""","""disease_wt""","""PTPRS""","""PTPRS""","""L08""","""B7A1R1_P4""","""B8A1R2_P4""","""disease_wt""",null,null,null,null,1.0,4,7,8,null,null,null,null,527.0,null,4,4,4,6,12,8,"""L""","""PTPRS""","""PTPRS"""
"""B8A1R2_P4""","""N08""","""allele""","""PTPRS""","""PTPRS_Pro548Leu""","""N08""","""B7A1R1_P4""","""B8A1R2_P4""","""allele""",null,null,null,"""Pro548Leu""",1.0,4,7,8,null,null,null,null,527.0,null,4,4,4,7,14,8,"""N""","""PTPRS""","""PTPRS"""
"""B8A1R2_P4""","""P08""","""disease_wt""","""TEAD2""","""TEAD2""","""P08""","""B7A1R1_P4""","""B8A1R2_P4""","""disease_wt""",null,null,null,null,1.0,4,7,8,null,null,null,null,527.0,null,4,4,4,8,16,8,"""P""","""TEAD2""","""TEAD2"""


In [104]:
plate_img_qc = pl.DataFrame()
well_img_qc_gfp_sum = pl.DataFrame()

for batch_id in BIO_REP_BATCHES:
    plate_img_qc = pl.concat([plate_img_qc, pl.read_parquet(f"{IMGS_QC_DIR}/{batch_id}/plate_well_sum_with_metrics.parquet")], how="diagonal_relaxed")
    plate_img_qc = plate_img_qc.with_columns(
        pl.col("plate").str.extract(r"([A-Za-z0-9]+_P\d+)T\d+", 1).alias("plate_id")
    ).with_columns(
        pl.concat_str([pl.col("plate_id"),pl.col("well")], separator="-").alias("plate_well")
    )#.filter((pl.col("plate").str.contains("B7A"))&(pl.col("channel")=="GFP"))
    allele_meta_df = allele_meta_dict[batch_id].with_columns(
        pl.concat_str([pl.col("plate_map_name"),pl.col("well_position")], separator="-").alias("plate_well")
    ).drop("plate","gene_symbol","symbol_original")
    plate_img_qc = plate_img_qc.join(
        allele_meta_df, 
        on="plate_well", how="left"
    ).filter(
        (pl.col("gene_allele").is_not_null())
    )
    # display(plate_img_qc)
    plate_img_qc_gfp = plate_img_qc.filter(pl.col("channel")=="GFP")
    display(plate_img_qc_gfp)
    
    well_img_qc_gfp_sum_batch = plate_img_qc_gfp.group_by(["gene_allele","well"]).agg(
        pl.col("is_bg").sum().alias("is_bg_sum"),
    ).with_columns(
        pl.lit(batch_id).alias("Batch"),
        pl.when(pl.col("is_bg_sum")>=3)
        .then(pl.lit("FAIL"))
        .otherwise(pl.lit("PASS"))
        .alias("QC_Flag")
    )
    display(well_img_qc_gfp_sum_batch)
    well_img_qc_gfp_sum = pl.concat([well_img_qc_gfp_sum, well_img_qc_gfp_sum_batch])

well_img_qc_gfp_sum.write_csv(f"{OUTPUT_DIR}/well-level_img_qc_gfp_sum.csv")

plate,well,channel,mean,std,perc_25,perc_50,perc_75,perc_80,perc_90,perc_95,perc_99,median_plate,mean_plate,std_plate,s2n_ratio,s2n_threshold,is_bg,plate_id,plate_well,plate_map_name,well_position,control_type,symbol,gene_allele,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,batch1,batch2,clone_ID,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,vector,notes,quad,c96,r96,r384,c384,r384_letter
str,str,str,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,bool,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,f64,i64,i64,i64,str,str,str,str,f64,str,i64,i64,i64,i64,i64,str
"""2024_01_23_B7A1R1_P4T2""","""O22""","""GFP""",150.944894,19.213892,138,153,165,168,174,179,190,158,279.635503,1000.696618,0.138906,0.179832,true,"""B7A1R1_P4""","""B7A1R1_P4-O22""","""B7A1R1_P4""","""O22""","""allele""","""TMEM43""","""TMEM43_Arg28Trp""","""O22""","""B7A1R1_P4""","""B8A1R2_P4""","""allele""",3438.0,201175.0,"""82C>T""","""Arg28Trp""",1.0,4,7,8,null,null,null,null,527.0,null,2,11,8,15,22,"""O"""
"""2024_01_23_B7A1R1_P4T2""","""M12""","""GFP""",153.210151,19.756116,140,155,167,170,177,182,193,158,279.635503,1000.696618,0.139461,0.179832,true,"""B7A1R1_P4""","""B7A1R1_P4-M12""","""B7A1R1_P4""","""M12""","""allele""","""TH""","""TH_Gly357Arg""","""M12""","""B7A1R1_P4""","""B8A1R2_P4""","""allele""",71892.0,4002.0,"""1069G>A""","""Gly357Arg""",1.0,4,7,8,null,null,null,null,527.0,null,2,6,7,13,12,"""M"""
"""2024_01_23_B7A1R1_P4T2""","""C18""","""GFP""",151.857203,19.438946,138,153,166,169,175,180,191,158,279.635503,1000.696618,0.141186,0.179832,true,"""B7A1R1_P4""","""B7A1R1_P4-C18""","""B7A1R1_P4""","""C18""","""allele""","""TH""","""TH_Arg200His""","""C18""","""B7A1R1_P4""","""B8A1R2_P4""","""allele""",71892.0,4017.0,"""599G>A""","""Arg200His""",1.0,4,7,8,null,null,null,null,527.0,null,2,9,2,3,18,"""C"""
"""2024_01_23_B7A1R1_P4T2""","""M05""","""GFP""",151.178747,19.423789,138,153,165,168,174,180,191,158,279.635503,1000.696618,0.141186,0.179832,true,"""B7A1R1_P4""","""B7A1R1_P4-M05""","""B7A1R1_P4""","""M05""","""allele""","""SMAD3""","""SMAD3_Pro124Leu""","""M05""","""B7A1R1_P4""","""B8A1R2_P4""","""allele""",11805.0,2127.0,"""371C>T""","""Pro124Leu""",1.0,4,7,8,null,null,null,null,527.0,null,1,3,7,13,5,"""M"""
"""2024_01_23_B7A1R1_P4T2""","""M08""","""GFP""",151.636989,19.470313,138,153,166,168,175,180,191,158,279.635503,1000.696618,0.141186,0.179832,true,"""B7A1R1_P4""","""B7A1R1_P4-M08""","""B7A1R1_P4""","""M08""","""allele""","""SUOX""","""SUOX_Ala265Asp""","""M08""","""B7A1R1_P4""","""B8A1R2_P4""","""allele""",71191.0,205704.0,"""794C>A""","""Ala265Asp""",1.0,4,7,8,null,null,null,null,527.0,null,2,4,7,13,8,"""M"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2024_01_17_B7A1R1_P2T3""","""N21""","""GFP""",4003.030408,9333.624194,169,200,3273,5088,11643,20815,53438,167,424.249085,2116.737413,2.499964,0.184106,false,"""B7A1R1_P2""","""B7A1R1_P2-N21""","""B7A1R1_P2""","""N21""","""allele""","""KLHL3""","""KLHL3_Glu85Ala""","""N21""","""B7A1R1_P2""","""B8A1R2_P2""","""allele""",1256.0,214009.0,"""254A>C""","""Glu85Ala""",1.0,2,7,8,null,null,null,null,527.0,null,3,11,7,14,21,"""N"""
"""2024_01_17_B7A1R1_P2T3""","""K23""","""GFP""",1191.104539,6981.838667,151,170,191,199,296,925,51664,167,424.249085,2116.737413,2.534211,0.184106,false,"""B7A1R1_P2""","""B7A1R1_P2-K23""","""B7A1R1_P2""","""K23""","""allele""","""GFAP""","""GFAP_Ala364Pro""","""K23""","""B7A1R1_P2""","""B8A1R2_P2""","""allele""",2920.0,214433.0,"""1090G>C""","""Ala364Pro""",1.0,2,7,8,null,null,null,null,527.0,null,1,12,6,11,23,"""K"""
"""2024_01_17_B7A1R1_P2T3""","""M21""","""GFP""",1351.61115,7350.872376,156,176,201,224,538,1538,56340,167,424.249085,2116.737413,2.557692,0.184106,false,"""B7A1R1_P2""","""B7A1R1_P2-M21""","""B7A1R1_P2""","""M21""","""allele""","""GFAP""","""GFAP_Arg376Trp""","""M21""","""B7A1R1_P2""","""

gene_allele,well,is_bg_sum,Batch,QC_Flag
str,str,u32,str,str
"""LITAF_Pro34Thr""","""D18""",0,"""2024_01_23_Batch_7""","""PASS"""
"""ACSF3_Thr358Ile""","""E01""",0,"""2024_01_23_Batch_7""","""PASS"""
"""SFTPC_Arg167Gln""","""F24""",0,"""2024_01_23_Batch_7""","""PASS"""
"""PKP2_Met365Ile""","""P07""",0,"""2024_01_23_Batch_7""","""PASS"""
"""KRT5""","""D02""",0,"""2024_01_23_Batch_7""","""PASS"""
…,…,…,…,…
"""GALE_Lys257Arg""","""O09""",0,"""2024_01_23_Batch_7""","""PASS"""
"""DDX20""","""H21""",0,"""2024_01_23_Batch_7""","""PASS"""
"""PKP2_Pro495Ala""","""B11""",0,"""2024_01_23_Batch_7""","""PASS"""


plate,well,channel,mean,std,perc_25,perc_50,perc_75,perc_80,perc_90,perc_95,perc_99,median_plate,mean_plate,std_plate,s2n_ratio,s2n_threshold,is_bg,plate_id,plate_well,plate_map_name,well_position,control_type,symbol,gene_allele,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,batch1,batch2,…,vector,notes,quad,c96,r96,r384,c384,r384_letter,plate_map_name_right,well_position_right,control_type_right,symbol_right,gene_allele_right,imaging_well_right,imaging_plate_R1_right,imaging_plate_R2_right,node_type_right,orf_id_wt_right,ccsb_mutation_id_right,nt_change_cdna_right,aa_change_right,allele_set_right,imaging_plate_right,batch1_right,batch2_right,clone_ID_right,hgmd_ID_right,clinvar_allele_ID_right,clinvar_clinical_significance_right,vector_right,notes_right,quad_right,c96_right,r96_right,r384_right,c384_right,r384_letter_right
str,str,str,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,bool,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,f64,i64,i64,i64,…,f64,str,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,f64,i64,i64,i64,str,str,str,str,f64,str,i64,i64,i64,i64,i64,str
"""2024_01_23_B7A1R1_P4T2""","""O22""","""GFP""",150.944894,19.213892,138,153,165,168,174,179,190,158,279.635503,1000.696618,0.138906,0.179832,true,"""B7A1R1_P4""","""B7A1R1_P4-O22""","""B7A1R1_P4""","""O22""","""allele""","""TMEM43""","""TMEM43_Arg28Trp""","""O22""","""B7A1R1_P4""","""B8A1R2_P4""","""allele""",3438.0,201175.0,"""82C>T""","""Arg28Trp""",1.0,4,7,8,…,527.0,null,2,11,8,15,22,"""O""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2024_01_23_B7A1R1_P4T2""","""M12""","""GFP""",153.210151,19.756116,140,155,167,170,177,182,193,158,279.635503,1000.696618,0.139461,0.179832,true,"""B7A1R1_P4""","""B7A1R1_P4-M12""","""B7A1R1_P4""","""M12""","""allele""","""TH""","""TH_Gly357Arg""","""M12""","""B7A1R1_P4""","""B8A1R2_P4""","""allele""",71892.0,4002.0,"""1069G>A""","""Gly357Arg""",1.0,4,7,8,…,527.0,null,2,6,7,13,12,"""M""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2024_01_23_B7A1R1_P4T2""","""C18""","""GFP""",151.857203,19.438946,138,153,166,169,175,180,191,158,279.635503,1000.696618,0.141186,0.179832,true,"""B7A1R1_P4""","""B7A1R1_P4-C18""","""B7A1R1_P4""","""C18""","""allele""","""TH""","""TH_Arg200His""","""C18""","""B7A1R1_P4""","""B8A1R2_P4""","""allele""",71892.0,4017.0,"""599G>A""","""Arg200His""",1.0,4,7,8,…,527.0,null,2,9,2,3,18,"""C""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2024_01_23_B7A1R1_P4T2""","""M05""","""GFP""",151.178747,19.423789,138,153,165,168,174,180,191,158,279.635503,1000.696618,0.141186,0.179832,true,"""B7A1R1_P4""","""B7A1R1_P4-M05""","""B7A1R1_P4""","""M05""","""allele""","""SMAD3""","""SMAD3_Pro124Leu""","""M05""","""B7A1R1_P4""","""B8A1R2_P4""","""allele""",11805.0,2127.0,"""371C>T""","""Pro124Leu""",1.0,4,7,8,…,527.0,null,1,3,7,13,5,"""M""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2024_01_23_B7A1R1_P4T2""","""M08""","""GFP""",151.636989,19.470313,138,153,166,168,175,180,191,158,279.635503,1000.696618,0.141186,0.179832,true,"""B7A1R1_P4""","""B7A1R1_P4-M08""","""B7A1R1_P4""","""M08""","""allele""","""SUOX""","""SUOX_Ala265Asp""","""M08""","""B7A1R1_P4""","""B8A1R2_P4""","""allele""",71191.0,205704.0,"""794C>A""","""Ala265Asp""",1.0,4,7,8,…,527.0,null,2,4,7,13,8,"""M""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…

gene_allele,well,is_bg_sum,Batch,QC_Flag
str,str,u32,str,str
"""PCBD1""","""M16""",0,"""2024_02_06_Batch_8""","""PASS"""
"""IL2RG_Trp237Arg""","""K24""",0,"""2024_02_06_Batch_8""","""PASS"""
"""PLA2G6_Thr585Ile""","""N13""",0,"""2024_02_06_Batch_8""","""PASS"""
"""MVK_Asn205Asp""","""A23""",0,"""2024_02_06_Batch_8""","""PASS"""
"""DCX_Thr203Ala""","""B17""",0,"""2024_02_06_Batch_8""","""PASS"""
…,…,…,…,…
"""PGBD1_Ile678Val""","""M18""",0,"""2024_02_06_Batch_8""","""PASS"""
"""AP2S1_Arg15Cys""","""E04""",0,"""2024_02_06_Batch_8""","""PASS"""
"""SMAD3_Thr261Ile""","""K15""",0,"""2024_02_06_Batch_8""","""PASS"""


## Batch 13-14

In [105]:
BIO_REP_BATCHES = ["2025_01_27_Batch_13", "2025_01_28_Batch_14"]
COMBINED_BIO_REP_DIR = "2025_01_Batch_13-14"
OUTPUT_DIR = f"../../outputs/{COMBINED_BIO_REP_DIR}"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [106]:
allele_meta_dict = {}
for batch_id in BIO_REP_BATCHES:
    allele_meta_df = pl.DataFrame()
    for platemap in os.listdir(PLATEMAP_DIR.format(batch_id=batch_id)):
        platemap_df = pl.read_csv(os.path.join(PLATEMAP_DIR.format(batch_id=batch_id), platemap), separator="\t", infer_schema_length=100000)
        allele_meta_df = pl.concat([allele_meta_df, 
                                    platemap_df.filter((~pl.col("node_type").is_in(["TC","NC","PC"]))&(pl.col("node_type").is_not_null()))], 
                                    how="diagonal_relaxed").sort("plate_map_name")
        allele_meta_df = allele_meta_df.with_columns(pl.col("plate_map_name").str.split('_').list.get(0).alias("plate_map"))
    allele_meta_dict[batch_id] = allele_meta_df
    display(allele_meta_df)#.sort()

plate_map_name,well_position,symbol,gene_allele,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,batch1,batch2,clone_ID,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,vector,notes,plate,quad,c96,r96,r384,c384,r384_letter,plate_map
str,str,str,str,str,str,str,str,f64,str,str,str,f64,f64,f64,f64,str,str,str,str,f64,str,i64,i64,i64,i64,i64,i64,str,str
"""B13A7A8P1_R1""","""A01""","""ABCD1""","""ABCD1""","""A01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""disease_wt""",1.00008637e8,null,null,null,7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,1,1,1,1,"""A""","""B13A7A8P1"""
"""B13A7A8P1_R1""","""C01""","""ABCD1""","""ABCD1_Arg518Trp""","""C01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""allele""",1.00008637e8,"""CCSBVarC007022""","""1552C>T""","""Arg518Trp""",7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,1,2,3,1,"""C""","""B13A7A8P1"""
"""B13A7A8P1_R1""","""E01""","""ABCD1""","""ABCD1_Arg389Gly""","""E01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""allele""",1.00008637e8,"""CCSBVarC007023""","""1165C>G""","""Arg389Gly""",7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,1,3,5,1,"""E""","""B13A7A8P1"""
"""B13A7A8P1_R1""","""G01""","""ABCD1""","""ABCD1_Arg518Gln""","""G01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""allele""",1.00008637e8,"""CCSBVarC007021""","""1553G>A""","""Arg518Gln""",7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,1,4,7,1,"""G""","""B13A7A8P1"""
"""B13A7A8P1_R1""","""I01""","""BAP1""","""BAP1""","""I01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""disease_wt""",3760.0,null,null,null,7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,1,5,9,1,"""I""","""B13A7A8P1"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""B13A7A8P2_R1""","""L08""","""AGXT""","""AGXT_Asp201Asn""","""L08""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""cPC""",null,null,null,"""Asp201Asn""",8.0,2.0,13.0,14.0,null,null,null,null,527.0,null,2,4,4,6,12,8,"""L""","""B13A7A8P2"""
"""B13A7A8P2_R1""","""P08""","""GMPPB""","""GMPPB""","""P08""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""cPC""",null,null,null,"""Cys164Phe""",null,2.0,13.0,14.0,null,null,null,null,527.0,null,2,4,4,8,16,8,"""P""","""B13A7A8P2"""
"""B13A7A8P2_R1""","""B10""","""GSS""","""GSS""","""B10""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""cPC""",null,null,null,null,8.0,2.0,13.0,14.0,null,null,null,null,527.0,null,2,4,5,1,2,10,"""B""","""B13A7A8P2"""


plate_map_name,well_position,symbol,gene_allele,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,batch1,batch2,clone_ID,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,vector,notes,plate,quad,c96,r96,r384,c384,r384_letter,plate_map
str,str,str,str,str,str,str,str,f64,str,str,str,f64,f64,f64,f64,str,str,str,str,f64,str,i64,i64,i64,i64,i64,i64,str,str
"""B14A7A8P1_R2""","""A01""","""ABCD1""","""ABCD1""","""A01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""disease_wt""",1.00008637e8,null,null,null,7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,1,1,1,1,"""A""","""B14A7A8P1"""
"""B14A7A8P1_R2""","""C01""","""ABCD1""","""ABCD1_Arg518Trp""","""C01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""allele""",1.00008637e8,"""CCSBVarC007022""","""1552C>T""","""Arg518Trp""",7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,1,2,3,1,"""C""","""B14A7A8P1"""
"""B14A7A8P1_R2""","""E01""","""ABCD1""","""ABCD1_Arg389Gly""","""E01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""allele""",1.00008637e8,"""CCSBVarC007023""","""1165C>G""","""Arg389Gly""",7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,1,3,5,1,"""E""","""B14A7A8P1"""
"""B14A7A8P1_R2""","""G01""","""ABCD1""","""ABCD1_Arg518Gln""","""G01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""allele""",1.00008637e8,"""CCSBVarC007021""","""1553G>A""","""Arg518Gln""",7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,1,4,7,1,"""G""","""B14A7A8P1"""
"""B14A7A8P1_R2""","""I01""","""BAP1""","""BAP1""","""I01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""disease_wt""",3760.0,null,null,null,7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,1,5,9,1,"""I""","""B14A7A8P1"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""B14A7A8P2_R2""","""L08""","""AGXT""","""AGXT_Asp201Asn""","""L08""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""cPC""",null,null,null,"""Asp201Asn""",8.0,2.0,13.0,14.0,null,null,null,null,527.0,null,2,4,4,6,12,8,"""L""","""B14A7A8P2"""
"""B14A7A8P2_R2""","""P08""","""GMPPB""","""GMPPB""","""P08""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""cPC""",null,null,null,"""Cys164Phe""",null,2.0,13.0,14.0,null,null,null,null,527.0,null,2,4,4,8,16,8,"""P""","""B14A7A8P2"""
"""B14A7A8P2_R2""","""B10""","""GSS""","""GSS""","""B10""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""cPC""",null,null,null,null,8.0,2.0,13.0,14.0,null,null,null,null,527.0,null,2,4,5,1,2,10,"""B""","""B14A7A8P2"""


In [107]:
plate_img_qc = pl.DataFrame()
well_img_qc_gfp_sum = pl.DataFrame()

for batch_id in BIO_REP_BATCHES:
    plate_img_qc = pl.concat([plate_img_qc, pl.read_parquet(f"{IMGS_QC_DIR}/{batch_id}/plate_well_sum_with_metrics.parquet")], how="diagonal_relaxed")
    plate_img_qc = plate_img_qc.with_columns(
        pl.col("plate").str.split('_').list.get(-2).alias("plate_id")
    ).with_columns(
        pl.concat_str([pl.col("plate_id"),pl.col("well")], separator="-").alias("plate_well")
    )#.filter((pl.col("plate").str.contains("B7A"))&(pl.col("channel")=="GFP"))
    display(plate_img_qc)

    allele_meta_df = allele_meta_dict[batch_id].with_columns(
        
        pl.concat_str([pl.col("plate_map"), pl.col("well_position")], separator="-").alias("plate_well")
    ).drop("plate","plate_map")
    display(allele_meta_df)
    plate_img_qc = plate_img_qc.join(
        allele_meta_df, 
        on="plate_well", how="left"
    ).filter(
        (pl.col("gene_allele").is_not_null())
    )
    # display(plate_img_qc)
    plate_img_qc_gfp = plate_img_qc.filter(pl.col("channel")=="GFP")
    display(plate_img_qc_gfp)
    
    well_img_qc_gfp_sum_batch = plate_img_qc_gfp.group_by(["gene_allele","well"]).agg(
        pl.col("is_bg").sum().alias("is_bg_sum"),
    ).with_columns(
        pl.lit(batch_id).alias("Batch"),
        pl.when(pl.col("is_bg_sum")>=3)
        .then(pl.lit("FAIL"))
        .otherwise(pl.lit("PASS"))
        .alias("QC_Flag")
    )
    display(well_img_qc_gfp_sum_batch)

    well_img_qc_gfp_sum = pl.concat([well_img_qc_gfp_sum, well_img_qc_gfp_sum_batch])

well_img_qc_gfp_sum.write_csv(f"{OUTPUT_DIR}/well-level_img_qc_gfp_sum.csv")

plate,well,channel,perc_25,perc_50,perc_75,perc_80,perc_90,perc_95,perc_99,mean,std,median_plate,mean_plate,std_plate,s2n_ratio,s2n_threshold,is_bg,plate_id,plate_well
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,bool,str,str
"""2025_01_27_B13A7A8P2_T4""","""E05""","""DAPI""",164.0,184.0,203.0,207.0,216.0,222.0,234.0,183.179462,41.330559,189,214.017741,173.470779,1.426829,1.570899,true,"""B13A7A8P2""","""B13A7A8P2-E05"""
"""2025_01_27_B13A7A8P2_T4""","""P12""","""DAPI""",173.0,194.0,213.0,217.0,227.0,234.0,247.0,193.046128,38.910745,189,214.017741,173.470779,1.427746,1.570899,true,"""B13A7A8P2""","""B13A7A8P2-P12"""
"""2025_01_27_B13A7A8P2_T4""","""F05""","""DAPI""",165.0,186.0,205.0,208.0,218.0,224.0,236.0,184.549546,46.0822,189,214.017741,173.470779,1.430303,1.570899,true,"""B13A7A8P2""","""B13A7A8P2-F05"""
"""2025_01_27_B13A7A8P2_T4""","""D18""","""DAPI""",165.0,186.0,205.0,209.0,218.0,225.0,237.0,185.080162,41.365839,189,214.017741,173.470779,1.436364,1.570899,true,"""B13A7A8P2""","""B13A7A8P2-D18"""
"""2025_01_27_B13A7A8P2_T4""","""I03""","""DAPI""",162.0,183.0,202.0,206.0,215.0,222.0,233.0,182.426163,58.407256,189,214.017741,173.470779,1.438272,1.570899,true,"""B13A7A8P2""","""B13A7A8P2-I03"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2025_01_27_B13A7A8P1_T4""","""D12""","""Mito""",429.0,514.0,592.0,637.0,1476.0,2497.0,6765.0,873.636807,1912.118352,547,765.519186,895.257204,15.769231,2.011954,false,"""B13A7A8P1""","""B13A7A8P1-D12"""
"""2025_01_27_B13A7A8P1_T4""","""N12""","""Mito""",429.0,500.0,558.0,573.0,753.0,1818.0,7038.0,741.84297,1469.195376,547,765.519186,895.257204,16.405594,2.011954,false,"""B13A7A8P1""","""B13A7A8P1-N12"""
"""2025_01_27_B13A7A8P1_T4""","""F12""","""Mito""",472.0,568.0,643.0,664.0,1231.0,2575.0,7918.0,893.453968,1778.899063,547,765.519186,895.257204,16.775424,2.011954,false,"""B13A7A8P1""","""B13A7A8P1-F12"""


plate_map_name,well_position,symbol,gene_allele,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,batch1,batch2,clone_ID,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,vector,notes,quad,c96,r96,r384,c384,r384_letter,plate_well
str,str,str,str,str,str,str,str,f64,str,str,str,f64,f64,f64,f64,str,str,str,str,f64,str,i64,i64,i64,i64,i64,str,str
"""B13A7A8P1_R1""","""A01""","""ABCD1""","""ABCD1""","""A01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""disease_wt""",1.00008637e8,null,null,null,7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,1,1,1,"""A""","""B13A7A8P1-A01"""
"""B13A7A8P1_R1""","""C01""","""ABCD1""","""ABCD1_Arg518Trp""","""C01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""allele""",1.00008637e8,"""CCSBVarC007022""","""1552C>T""","""Arg518Trp""",7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,2,3,1,"""C""","""B13A7A8P1-C01"""
"""B13A7A8P1_R1""","""E01""","""ABCD1""","""ABCD1_Arg389Gly""","""E01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""allele""",1.00008637e8,"""CCSBVarC007023""","""1165C>G""","""Arg389Gly""",7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,3,5,1,"""E""","""B13A7A8P1-E01"""
"""B13A7A8P1_R1""","""G01""","""ABCD1""","""ABCD1_Arg518Gln""","""G01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""allele""",1.00008637e8,"""CCSBVarC007021""","""1553G>A""","""Arg518Gln""",7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,4,7,1,"""G""","""B13A7A8P1-G01"""
"""B13A7A8P1_R1""","""I01""","""BAP1""","""BAP1""","""I01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""disease_wt""",3760.0,null,null,null,7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,5,9,1,"""I""","""B13A7A8P1-I01"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""B13A7A8P2_R1""","""L08""","""AGXT""","""AGXT_Asp201Asn""","""L08""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""cPC""",null,null,null,"""Asp201Asn""",8.0,2.0,13.0,14.0,null,null,null,null,527.0,null,4,4,6,12,8,"""L""","""B13A7A8P2-L08"""
"""B13A7A8P2_R1""","""P08""","""GMPPB""","""GMPPB""","""P08""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""cPC""",null,null,null,"""Cys164Phe""",null,2.0,13.0,14.0,null,null,null,null,527.0,null,4,4,8,16,8,"""P""","""B13A7A8P2-P08"""
"""B13A7A8P2_R1""","""B10""","""GSS""","""GSS""","""B10""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""cPC""",null,null,null,null,8.0,2.0,13.0,14.0,null,null,null,null,527.0,null,4,5,1,2,10,"""B""","""B13A7A8P2-B10"""


plate,well,channel,perc_25,perc_50,perc_75,perc_80,perc_90,perc_95,perc_99,mean,std,median_plate,mean_plate,std_plate,s2n_ratio,s2n_threshold,is_bg,plate_id,plate_well,plate_map_name,well_position,symbol,gene_allele,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,batch1,batch2,clone_ID,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,vector,notes,quad,c96,r96,r384,c384,r384_letter
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,bool,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,f64,f64,f64,f64,str,str,str,str,f64,str,i64,i64,i64,i64,i64,str
"""2025_01_27_B13A7A8P2_T4""","""G22""","""GFP""",138.0,153.0,166.0,169.0,175.0,181.0,190.0,151.630641,20.891313,170,963.29211,3250.858319,0.138906,0.184974,true,"""B13A7A8P2""","""B13A7A8P2-G22""","""B13A7A8P2_R1""","""G22""","""CCM2""","""CCM2_Glu108Lys""","""G22""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""allele""",3928.0,"""CCSBVarC038649""","""322G>A""","""Glu108Lys""",8.0,2.0,13.0,14.0,null,null,null,null,527.0,null,2,11,4,7,22,"""G"""
"""2025_01_27_B13A7A8P2_T4""","""N09""","""GFP""",141.0,157.0,171.0,174.0,181.0,186.0,196.0,155.813646,21.977332,170,963.29211,3250.858319,0.143068,0.184974,true,"""B13A7A8P2""","""B13A7A8P2-N09""","""B13A7A8P2_R1""","""N09""","""CCM2""","""CCM2_Phe217Leu""","""N09""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""allele""",3928.0,"""CCSBVarC038723""","""649T>C""","""Phe217Leu""",8.0,2.0,13.0,14.0,null,null,null,null,527.0,null,3,5,7,14,9,"""N"""
"""2025_01_27_B13A7A8P2_T4""","""E04""","""GFP""",137.0,153.0,166.0,169.0,176.0,181.0,192.0,158.593228,563.841808,170,963.29211,3250.858319,0.146612,0.184974,true,"""B13A7A8P2""","""B13A7A8P2-E04""","""B13A7A8P2_R1""","""E04""","""SHOC2""","""SHOC2_Ser57Phe""","""E04""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""allele""",11602.0,"""CCSBVarC028011""","""170C>T""","""Ser57Phe""",7.0,2.0,13.0,14.0,null,null,null,null,527.0,null,2,2,3,5,4,"""E"""
"""2025_01_27_B13A7A8P2_T4""","""I03""","""GFP""",137.0,153.0,167.0,170.0,177.0,183.0,192.0,151.63554,20.364756,170,963.29211,3250.858319,0.146612,0.184974,true,"""B13A7A8P2""","""B13A7A8P2-I03""","""B13A7A8P2_R1""","""I03""","""RAD51D""","""RAD51D_Phe305Leu""","""I03""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""allele""",7201.0,"""CCSBVarC024886""","""915C>G""","""Phe305Leu""",7.0,2.0,13.0,14.0,null,null,null,null,527.0,null,1,2,5,9,3,"""I"""
"""2025_01_27_B13A7A8P2_T4""","""I04""","""GFP""",137.0,153.0,167.0,170.0,177.0,182.0,192.0,152.803773,54.302171,170,963.29211,3250.858319,0.146612,0.184974,true,"""B13A7A8P2""","""B13A7A8P2-I04""","""B13A7A8P2_R1""","""I04""","""SHOC2""","""SHOC2_Met173Val""","""I04""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""allele""",11602.0,"""CCSBVarC028030""","""517A>G""","""Met173Val""",7.0,2.0,13.0,14.0,null,null,null,null,527.0,null,2,2,5,9,4,"""I"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2025_01_27_B13A7A8P1_T4""","""N12""","""GFP""",152.0,172.0,196.0,205.0,331.0,888.0,58038.0,1150.12622,7081.219744,169,578.298608,3064.646236,2.581869,0.187358,false,"""B13A7A8P1""","""B13A7A8P1-N12""","""B13A7A8P1_R1""","""N12""","""NF2""","""NF2_Lys206Glu""","""N12""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""allele""",3697.0,"""CCSBVarC016645""","""616A>G""","""Lys206Glu""",7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,4,6,7,14,12,"""N"""
"""2025_01_27_B13A7A8P1_T4""","""J07""","""GFP""",169.0,201.0,3683.0,6350.0,16833.0,31551.0,65535.0,5250.123131,12082.218232,169,578.298608,3064.646236,2.588587,0.187358,false,"""B13A7A8P1""","""B13A7A8P1-J07""","""B13A7A8P1_R1""","""J07""","""G6PD""","""G6PD""","""J07""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""disease_wt""",4006.0,null,null,null,7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,3,4,5,10,7,"""J"""
"""2025_01_27_B13A7A8P1_T4""","""F12""","""GFP""",155.0,176.0,200.0,214.0,537.0,1201.0,60594.0,1221.095513,7239.668714,169,578.298608,3

gene_allele,well,is_bg_sum,Batch,QC_Flag
str,str,u32,str,str
"""CCM2_Ala123Val""","""A24""",0,"""2025_01_27_Batch_13""","""PASS"""
"""ZC4H2_Pro201Ser""","""K06""",0,"""2025_01_27_Batch_13""","""PASS"""
"""CCM2_Cys211Ser""","""D09""",0,"""2025_01_27_Batch_13""","""PASS"""
"""SDHD_Leu139Arg""","""G02""",0,"""2025_01_27_Batch_13""","""PASS"""
"""CCM2_Thr222Met""","""D11""",0,"""2025_01_27_Batch_13""","""PASS"""
…,…,…,…,…
"""BRCA1_Cys44Tyr""","""M11""",0,"""2025_01_27_Batch_13""","""PASS"""
"""CCM2_Glu433Lys""","""L06""",0,"""2025_01_27_Batch_13""","""PASS"""
"""SHOC2_Thr321Ala""","""P17""",0,"""2025_01_27_Batch_13""","""PASS"""


plate,well,channel,perc_25,perc_50,perc_75,perc_80,perc_90,perc_95,perc_99,mean,std,median_plate,mean_plate,std_plate,s2n_ratio,s2n_threshold,is_bg,plate_id,plate_well,plate_map_name,well_position,symbol,gene_allele,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,batch1,batch2,clone_ID,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,vector,notes,quad,c96,r96,r384,c384,r384_letter
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,bool,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,f64,f64,f64,f64,str,str,str,str,f64,str,i64,i64,i64,i64,i64,str
"""2025_01_27_B13A7A8P2_T4""","""I03""","""DAPI""",162.0,183.0,202.0,206.0,215.0,222.0,233.0,182.426163,58.407256,189,214.017741,173.470779,1.438272,1.570899,true,"""B13A7A8P2""","""B13A7A8P2-I03""","""B13A7A8P2_R1""","""I03""","""RAD51D""","""RAD51D_Phe305Leu""","""I03""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""allele""",7201.0,"""CCSBVarC024886""","""915C>G""","""Phe305Leu""",7.0,2.0,13.0,14.0,null,null,null,null,527.0,null,1,2,5,9,3,"""I"""
"""2025_01_27_B13A7A8P2_T4""","""N09""","""DAPI""",169.0,191.0,210.0,214.0,224.0,231.0,244.0,190.086991,54.134039,189,214.017741,173.470779,1.443787,1.570899,true,"""B13A7A8P2""","""B13A7A8P2-N09""","""B13A7A8P2_R1""","""N09""","""CCM2""","""CCM2_Phe217Leu""","""N09""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""allele""",3928.0,"""CCSBVarC038723""","""649T>C""","""Phe217Leu""",8.0,2.0,13.0,14.0,null,null,null,null,527.0,null,3,5,7,14,9,"""N"""
"""2025_01_27_B13A7A8P2_T4""","""I04""","""DAPI""",162.0,183.0,201.0,205.0,214.0,221.0,235.0,183.258259,114.750291,189,214.017741,173.470779,1.450617,1.570899,true,"""B13A7A8P2""","""B13A7A8P2-I04""","""B13A7A8P2_R1""","""I04""","""SHOC2""","""SHOC2_Met173Val""","""I04""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""allele""",11602.0,"""CCSBVarC028030""","""517A>G""","""Met173Val""",7.0,2.0,13.0,14.0,null,null,null,null,527.0,null,2,2,5,9,4,"""I"""
"""2025_01_27_B13A7A8P2_T4""","""E04""","""DAPI""",162.0,183.0,201.0,205.0,214.0,221.0,237.0,183.510139,66.388183,189,214.017741,173.470779,1.462963,1.570899,true,"""B13A7A8P2""","""B13A7A8P2-E04""","""B13A7A8P2_R1""","""E04""","""SHOC2""","""SHOC2_Ser57Phe""","""E04""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""allele""",11602.0,"""CCSBVarC028011""","""170C>T""","""Ser57Phe""",7.0,2.0,13.0,14.0,null,null,null,null,527.0,null,2,2,3,5,4,"""E"""
"""2025_01_27_B13A7A8P2_T4""","""G22""","""DAPI""",161.0,182.0,200.0,204.0,213.0,220.0,237.0,182.65232,77.560333,189,214.017741,173.470779,1.47205,1.570899,true,"""B13A7A8P2""","""B13A7A8P2-G22""","""B13A7A8P2_R1""","""G22""","""CCM2""","""CCM2_Glu108Lys""","""G22""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""allele""",3928.0,"""CCSBVarC038649""","""322G>A""","""Glu108Lys""",8.0,2.0,13.0,14.0,null,null,null,null,527.0,null,2,11,4,7,22,"""G"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2025_01_28_B14A7A8P2_T2""","""H08""","""Mito""",436.0,538.0,663.0,902.0,1954.0,2977.0,5534.0,877.960759,1044.117341,536,777.415025,838.755687,12.692661,1.751275,false,"""B14A7A8P2""","""B14A7A8P2-H08""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2025_01_28_B14A7A8P2_T2""","""E11""","""Mito""",448.0,545.0,624.0,644.0,1028.0,2378.0,5919.0,831.203641,1878.601783,536,777.415025,838.755687,13.212054,1.751275,false,"""B14A7A8P2""","""B14A7A8P2-E11""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2025_01_28_B14A7A8P2_T2""","""F19""","""Mito""",510.0,608.0,706.0,776.0,2088.0,3420.0,6746.0,1003.190105,1459.912983,536,777.415025,838.755687,13.227451,1.751275,false,"""B14A7A8P2""","""B14A7A8P2-F19""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,

plate_map_name,well_position,symbol,gene_allele,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,batch1,batch2,clone_ID,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,vector,notes,quad,c96,r96,r384,c384,r384_letter,plate_well
str,str,str,str,str,str,str,str,f64,str,str,str,f64,f64,f64,f64,str,str,str,str,f64,str,i64,i64,i64,i64,i64,str,str
"""B14A7A8P1_R2""","""A01""","""ABCD1""","""ABCD1""","""A01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""disease_wt""",1.00008637e8,null,null,null,7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,1,1,1,"""A""","""B14A7A8P1-A01"""
"""B14A7A8P1_R2""","""C01""","""ABCD1""","""ABCD1_Arg518Trp""","""C01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""allele""",1.00008637e8,"""CCSBVarC007022""","""1552C>T""","""Arg518Trp""",7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,2,3,1,"""C""","""B14A7A8P1-C01"""
"""B14A7A8P1_R2""","""E01""","""ABCD1""","""ABCD1_Arg389Gly""","""E01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""allele""",1.00008637e8,"""CCSBVarC007023""","""1165C>G""","""Arg389Gly""",7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,3,5,1,"""E""","""B14A7A8P1-E01"""
"""B14A7A8P1_R2""","""G01""","""ABCD1""","""ABCD1_Arg518Gln""","""G01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""allele""",1.00008637e8,"""CCSBVarC007021""","""1553G>A""","""Arg518Gln""",7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,4,7,1,"""G""","""B14A7A8P1-G01"""
"""B14A7A8P1_R2""","""I01""","""BAP1""","""BAP1""","""I01""","""B13A7A8P1_R1""","""B14A7A8P1_R2""","""disease_wt""",3760.0,null,null,null,7.0,1.0,13.0,14.0,null,null,null,null,527.0,null,1,1,5,9,1,"""I""","""B14A7A8P1-I01"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""B14A7A8P2_R2""","""L08""","""AGXT""","""AGXT_Asp201Asn""","""L08""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""cPC""",null,null,null,"""Asp201Asn""",8.0,2.0,13.0,14.0,null,null,null,null,527.0,null,4,4,6,12,8,"""L""","""B14A7A8P2-L08"""
"""B14A7A8P2_R2""","""P08""","""GMPPB""","""GMPPB""","""P08""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""cPC""",null,null,null,"""Cys164Phe""",null,2.0,13.0,14.0,null,null,null,null,527.0,null,4,4,8,16,8,"""P""","""B14A7A8P2-P08"""
"""B14A7A8P2_R2""","""B10""","""GSS""","""GSS""","""B10""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""cPC""",null,null,null,null,8.0,2.0,13.0,14.0,null,null,null,null,527.0,null,4,5,1,2,10,"""B""","""B14A7A8P2-B10"""


plate,well,channel,perc_25,perc_50,perc_75,perc_80,perc_90,perc_95,perc_99,mean,std,median_plate,mean_plate,std_plate,s2n_ratio,s2n_threshold,is_bg,plate_id,plate_well,plate_map_name,well_position,symbol,gene_allele,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,batch1,batch2,clone_ID,…,clinvar_clinical_significance,vector,notes,quad,c96,r96,r384,c384,r384_letter,plate_map_name_right,well_position_right,symbol_right,gene_allele_right,imaging_well_right,imaging_plate_R1_right,imaging_plate_R2_right,node_type_right,orf_id_wt_right,ccsb_mutation_id_right,nt_change_cdna_right,aa_change_right,allele_set_right,imaging_plate_right,batch1_right,batch2_right,clone_ID_right,hgmd_ID_right,clinvar_allele_ID_right,clinvar_clinical_significance_right,vector_right,notes_right,quad_right,c96_right,r96_right,r384_right,c384_right,r384_letter_right
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,bool,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,f64,f64,f64,f64,str,…,str,f64,str,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,f64,str,str,str,f64,f64,f64,f64,str,str,str,str,f64,str,i64,i64,i64,i64,i64,str
"""2025_01_27_B13A7A8P2_T4""","""G22""","""GFP""",138.0,153.0,166.0,169.0,175.0,181.0,190.0,151.630641,20.891313,170,963.29211,3250.858319,0.138906,0.184974,true,"""B13A7A8P2""","""B13A7A8P2-G22""","""B13A7A8P2_R1""","""G22""","""CCM2""","""CCM2_Glu108Lys""","""G22""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""allele""",3928.0,"""CCSBVarC038649""","""322G>A""","""Glu108Lys""",8.0,2.0,13.0,14.0,null,…,null,527.0,null,2,11,4,7,22,"""G""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2025_01_27_B13A7A8P2_T4""","""N09""","""GFP""",141.0,157.0,171.0,174.0,181.0,186.0,196.0,155.813646,21.977332,170,963.29211,3250.858319,0.143068,0.184974,true,"""B13A7A8P2""","""B13A7A8P2-N09""","""B13A7A8P2_R1""","""N09""","""CCM2""","""CCM2_Phe217Leu""","""N09""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""allele""",3928.0,"""CCSBVarC038723""","""649T>C""","""Phe217Leu""",8.0,2.0,13.0,14.0,null,…,null,527.0,null,3,5,7,14,9,"""N""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2025_01_27_B13A7A8P2_T4""","""E04""","""GFP""",137.0,153.0,166.0,169.0,176.0,181.0,192.0,158.593228,563.841808,170,963.29211,3250.858319,0.146612,0.184974,true,"""B13A7A8P2""","""B13A7A8P2-E04""","""B13A7A8P2_R1""","""E04""","""SHOC2""","""SHOC2_Ser57Phe""","""E04""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""allele""",11602.0,"""CCSBVarC028011""","""170C>T""","""Ser57Phe""",7.0,2.0,13.0,14.0,null,…,null,527.0,null,2,2,3,5,4,"""E""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2025_01_27_B13A7A8P2_T4""","""I03""","""GFP""",137.0,153.0,167.0,170.0,177.0,183.0,192.0,151.63554,20.364756,170,963.29211,3250.858319,0.146612,0.184974,true,"""B13A7A8P2""","""B13A7A8P2-I03""","""B13A7A8P2_R1""","""I03""","""RAD51D""","""RAD51D_Phe305Leu""","""I03""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""allele""",7201.0,"""CCSBVarC024886""","""915C>G""","""Phe305Leu""",7.0,2.0,13.0,14.0,null,…,null,527.0,null,1,2,5,9,3,"""I""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2025_01_27_B13A7A8P2_T4""","""I04""","""GFP""",137.0,153.0,167.0,170.0,177.0,182.0,192.0,152.803773,54.302171,170,963.29211,3250.858319,0.146612,0.184974,true,"""B13A7A8P2""","""B13A7A8P2-I04""","""B13A7A8P2_R1""","""I04""","""SHOC2""","""SHOC2_Met173Val""","""I04""","""B13A7A8P2_R1""","""B14A7A8P2_R2""","""allele""",11602.0,"""CCSBVarC028030""","""517A>G""","""Met173Val""",7.0,2.0,13.0,14.0,null,…,null,527.0,null,2,2,5,9,4,"""I""",null,null,null,null,

gene_allele,well,is_bg_sum,Batch,QC_Flag
str,str,u32,str,str
"""SHOC2_Met173Val""","""L19""",0,"""2025_01_28_Batch_14""","""PASS"""
"""SDHD""","""A21""",0,"""2025_01_28_Batch_14""","""PASS"""
"""ZC4H2_Ala162Thr""","""C08""",0,"""2025_01_28_Batch_14""","""PASS"""
"""F9_Val257Ile""","""I18""",0,"""2025_01_28_Batch_14""","""PASS"""
"""SHOC2_Thr205Ala""","""K04""",4,"""2025_01_28_Batch_14""","""FAIL"""
…,…,…,…,…
"""F9_Cys178Phe""","""A16""",0,"""2025_01_28_Batch_14""","""PASS"""
"""SHOC2_Thr494Asn""","""L21""",0,"""2025_01_28_Batch_14""","""PASS"""
"""CCM2_Gly379Ser""","""L21""",0,"""2025_01_28_Batch_14""","""PASS"""


## Batch 15-16

In [108]:
BIO_REP_BATCHES = ["2025_03_17_Batch_15", "2025_03_17_Batch_16"]
COMBINED_BIO_REP_DIR = "2025_03_Batch_15-16"

OUTPUT_DIR = f"../../outputs/{COMBINED_BIO_REP_DIR}"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [109]:
allele_meta_dict = {}
for batch_id in BIO_REP_BATCHES:
    allele_meta_df = pl.DataFrame()
    for platemap in os.listdir(PLATEMAP_DIR.format(batch_id=batch_id)):
        platemap_df = pl.read_csv(os.path.join(PLATEMAP_DIR.format(batch_id=batch_id), platemap), separator="\t", infer_schema_length=100000)
        allele_meta_df = pl.concat([allele_meta_df, 
                                    platemap_df.filter((~pl.col("node_type").is_in(["TC","NC","PC"]))&(pl.col("node_type").is_not_null()))], 
                                    how="diagonal_relaxed").sort("plate_map_name")
        allele_meta_df = allele_meta_df.with_columns(pl.col("plate_map_name").str.split('_').list.get(0).alias("plate_map"))
    allele_meta_dict[batch_id] = allele_meta_df
    display(allele_meta_df)#.sort()

plate_map_name,well_position,symbol,gene_allele,source_plate,source_well,source_numb,dest_plate,dest_well,dest_numb,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,nt_change_cdna,aa_change,ccsb_mutation_id,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,sequencing_confidence,plate_map
str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,str,str,f64,str,str,str,str,str
"""B15A1A2P1_R1""","""A01""","""AP2S1""","""AP2S1""","""FV_reLR_01""","""G01""",7.0,"""FVrry_P05_Q01""","""A01""",1,"""A01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",4983.0,null,null,null,null,null,null,"""high ""","""B15A1A2P1"""
"""B15A1A2P1_R1""","""C01""","""AP2S1""","""AP2S1_Arg15Cys""","""VUSmut_GDEh06""","""B04""",26.0,"""FVrry_P05_Q01""","""B01""",2,"""C01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4983.0,"""43C>T""","""Arg15Cys""",215314.0,null,null,null,null,"""B15A1A2P1"""
"""B15A1A2P1_R1""","""E01""","""AP2S1""","""AP2S1_Arg15His""","""VUSmut_GDEh06""","""G05""",39.0,"""FVrry_P05_Q01""","""C01""",3,"""E01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4983.0,"""44G>A""","""Arg15His""",215316.0,null,null,null,null,"""B15A1A2P1"""
"""B15A1A2P1_R1""","""G01""","""APOA1""","""APOA1""","""DisWTGDEh03""","""C05""",35.0,"""FVrry_P05_Q01""","""D01""",4,"""G01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",5613.0,null,null,null,null,null,null,null,"""B15A1A2P1"""
"""B15A1A2P1_R1""","""I01""","""APOA1""","""APOA1_Arg34Leu""","""VUSmut_GDEh09""","""C12""",91.0,"""FVrry_P05_Q01""","""E01""",5,"""I01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",5613.0,"""101G>T""","""Arg34Leu""",215445.0,null,null,null,null,"""B15A1A2P1"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""B15A1A2P1_R1""","""D08""","""TPM1""","""TPM1_Glu40Lys""","""VUSmut_GDEh12""","""G11""",87.0,"""FVrry_P05_Q04""","""B04""",26,"""D08""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4723.0,"""118G>A""","""Glu40Lys""",8945.0,null,null,null,null,"""B15A1A2P1"""
"""B15A1A2P1_R1""","""F08""","""TPM1""","""TPM1_Val95Ala""","""VUSmut_GDEh12""","""H11""",88.0,"""FVrry_P05_Q04""","""C04""",27,"""F08""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4723.0,"""284T>C""","""Val95Ala""",8950.0,null,null,null,null,"""B15A1A2P1"""
"""B15A1A2P1_R1""","""J08""","""TPM3""","""TPM3""","""DisWTGDEh03""","""F07""",54.0,"""FVrry_P05_Q04""","""E04""",29,"""J08""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",5712.0,null,null,null,null,null,null,null,"""B15A1A2P1"""


plate_map_name,well_position,symbol,gene_allele,source_plate,source_well,source_numb,dest_plate,dest_well,dest_numb,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,nt_change_cdna,aa_change,ccsb_mutation_id,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,sequencing_confidence,plate_map
str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,str,str,f64,str,str,str,str,str
"""B16A1A2P1_R1""","""A01""","""AP2S1""","""AP2S1""","""FV_reLR_01""","""G01""",7.0,"""FVrry_P05_Q01""","""A01""",1,"""A01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",4983.0,null,null,null,null,null,null,"""high ""","""B16A1A2P1"""
"""B16A1A2P1_R1""","""C01""","""AP2S1""","""AP2S1_Arg15Cys""","""VUSmut_GDEh06""","""B04""",26.0,"""FVrry_P05_Q01""","""B01""",2,"""C01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4983.0,"""43C>T""","""Arg15Cys""",215314.0,null,null,null,null,"""B16A1A2P1"""
"""B16A1A2P1_R1""","""E01""","""AP2S1""","""AP2S1_Arg15His""","""VUSmut_GDEh06""","""G05""",39.0,"""FVrry_P05_Q01""","""C01""",3,"""E01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4983.0,"""44G>A""","""Arg15His""",215316.0,null,null,null,null,"""B16A1A2P1"""
"""B16A1A2P1_R1""","""G01""","""APOA1""","""APOA1""","""DisWTGDEh03""","""C05""",35.0,"""FVrry_P05_Q01""","""D01""",4,"""G01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",5613.0,null,null,null,null,null,null,null,"""B16A1A2P1"""
"""B16A1A2P1_R1""","""I01""","""APOA1""","""APOA1_Arg34Leu""","""VUSmut_GDEh09""","""C12""",91.0,"""FVrry_P05_Q01""","""E01""",5,"""I01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",5613.0,"""101G>T""","""Arg34Leu""",215445.0,null,null,null,null,"""B16A1A2P1"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""B16A1A2P1_R1""","""D08""","""TPM1""","""TPM1_Glu40Lys""","""VUSmut_GDEh12""","""G11""",87.0,"""FVrry_P05_Q04""","""B04""",26,"""D08""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4723.0,"""118G>A""","""Glu40Lys""",8945.0,null,null,null,null,"""B16A1A2P1"""
"""B16A1A2P1_R1""","""F08""","""TPM1""","""TPM1_Val95Ala""","""VUSmut_GDEh12""","""H11""",88.0,"""FVrry_P05_Q04""","""C04""",27,"""F08""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4723.0,"""284T>C""","""Val95Ala""",8950.0,null,null,null,null,"""B16A1A2P1"""
"""B16A1A2P1_R1""","""J08""","""TPM3""","""TPM3""","""DisWTGDEh03""","""F07""",54.0,"""FVrry_P05_Q04""","""E04""",29,"""J08""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",5712.0,null,null,null,null,null,null,null,"""B16A1A2P1"""


In [110]:
plate_img_qc = pl.DataFrame()
well_img_qc_gfp_sum = pl.DataFrame()

for batch_id in BIO_REP_BATCHES:
    plate_img_qc = pl.concat([plate_img_qc, pl.read_parquet(f"{IMGS_QC_DIR}/{batch_id}/plate_well_sum_with_metrics.parquet")], how="diagonal_relaxed")
    plate_img_qc = plate_img_qc.with_columns(
        pl.col("plate").str.extract(r"([A-Za-z0-9]+_P\d+)T\d+", 1).str.replace("_", "").alias("plate_id")
    ).with_columns(
        pl.concat_str([pl.col("plate_id"),pl.col("well")], separator="-").alias("plate_well")
    )#.filter((pl.col("plate").str.contains("B7A"))&(pl.col("channel")=="GFP"))
    display(plate_img_qc)

    allele_meta_df = allele_meta_dict[batch_id].with_columns(
        pl.concat_str([pl.col("plate_map"), pl.col("well_position")], separator="-").alias("plate_well")
    ).drop("plate_map")
    display(allele_meta_df)
    plate_img_qc = plate_img_qc.join(
        allele_meta_df, 
        on="plate_well", how="left"
    ).filter(
        (pl.col("gene_allele").is_not_null())
    )
    # display(plate_img_qc)
    plate_img_qc_gfp = plate_img_qc.filter(pl.col("channel")=="GFP")
    display(plate_img_qc_gfp)
    
    well_img_qc_gfp_sum_batch = plate_img_qc_gfp.group_by(["gene_allele","well"]).agg(
        pl.col("is_bg").sum().alias("is_bg_sum"),
    ).with_columns(
        pl.lit(batch_id).alias("Batch"),
        pl.when(pl.col("is_bg_sum")>=3)
        .then(pl.lit("FAIL"))
        .otherwise(pl.lit("PASS"))
        .alias("QC_Flag")
    )
    display(well_img_qc_gfp_sum_batch)
    

    well_img_qc_gfp_sum = pl.concat([well_img_qc_gfp_sum, well_img_qc_gfp_sum_batch])

well_img_qc_gfp_sum.write_csv(f"{OUTPUT_DIR}/well-level_img_qc_gfp_sum.csv")

plate,well,channel,perc_25,perc_50,perc_75,perc_80,perc_90,perc_95,perc_99,mean,std,median_plate,mean_plate,std_plate,s2n_ratio,s2n_threshold,is_bg,plate_id,plate_well
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,bool,str,str
"""2025-03-17_B15A1A2_P1T4""","""D07""","""DAPI""",161.0,180.0,197.0,201.0,209.0,216.0,226.0,178.472598,24.98599,180,188.175043,68.561838,1.403727,1.454161,true,"""B15A1A2P1""","""B15A1A2P1-D07"""
"""2025-03-17_B15A1A2_P1T4""","""P12""","""DAPI""",165.0,185.0,202.0,206.0,214.0,221.0,232.0,183.506252,31.738052,180,188.175043,68.561838,1.406061,1.454161,true,"""B15A1A2P1""","""B15A1A2P1-P12"""
"""2025-03-17_B15A1A2_P1T4""","""P24""","""DAPI""",154.0,172.0,188.0,191.0,199.0,205.0,217.0,171.234885,40.787681,180,188.175043,68.561838,1.409091,1.454161,true,"""B15A1A2P1""","""B15A1A2P1-P24"""
"""2025-03-17_B15A1A2_P1T4""","""N16""","""DAPI""",163.0,183.0,200.0,204.0,213.0,219.0,230.0,181.333759,30.731548,180,188.175043,68.561838,1.411043,1.454161,true,"""B15A1A2P1""","""B15A1A2P1-N16"""
"""2025-03-17_B15A1A2_P1T4""","""E05""","""DAPI""",158.0,177.0,194.0,197.0,206.0,212.0,223.0,175.585432,32.297659,180,188.175043,68.561838,1.411392,1.454161,true,"""B15A1A2P1""","""B15A1A2P1-E05"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2025-03-17_B15A1A2_P1T3""","""O06""","""Mito""",1020.0,1570.0,4367.0,5462.0,8523.0,11058.0,17029.0,3353.046799,3724.608499,431,654.399456,848.138254,16.695098,1.839439,false,"""B15A1A2P1""","""B15A1A2P1-O06"""
"""2025-03-17_B15A1A2_P1T3""","""O03""","""Mito""",779.0,1008.0,1228.0,1325.0,3583.0,6682.0,13029.0,1721.322592,2709.483338,431,654.399456,848.138254,16.725289,1.839439,false,"""B15A1A2P1""","""B15A1A2P1-O03"""
"""2025-03-17_B15A1A2_P1T3""","""O02""","""Mito""",953.0,1290.0,3703.0,4726.0,7920.0,10472.0,15989.0,2984.953689,3526.543957,431,654.399456,848.138254,16.777545,1.839439,false,"""B15A1A2P1""","""B15A1A2P1-O02"""


plate_map_name,well_position,symbol,gene_allele,source_plate,source_well,source_numb,dest_plate,dest_well,dest_numb,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,nt_change_cdna,aa_change,ccsb_mutation_id,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,sequencing_confidence,plate_well
str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,str,str,f64,str,str,str,str,str
"""B15A1A2P1_R1""","""A01""","""AP2S1""","""AP2S1""","""FV_reLR_01""","""G01""",7.0,"""FVrry_P05_Q01""","""A01""",1,"""A01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",4983.0,null,null,null,null,null,null,"""high ""","""B15A1A2P1-A01"""
"""B15A1A2P1_R1""","""C01""","""AP2S1""","""AP2S1_Arg15Cys""","""VUSmut_GDEh06""","""B04""",26.0,"""FVrry_P05_Q01""","""B01""",2,"""C01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4983.0,"""43C>T""","""Arg15Cys""",215314.0,null,null,null,null,"""B15A1A2P1-C01"""
"""B15A1A2P1_R1""","""E01""","""AP2S1""","""AP2S1_Arg15His""","""VUSmut_GDEh06""","""G05""",39.0,"""FVrry_P05_Q01""","""C01""",3,"""E01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4983.0,"""44G>A""","""Arg15His""",215316.0,null,null,null,null,"""B15A1A2P1-E01"""
"""B15A1A2P1_R1""","""G01""","""APOA1""","""APOA1""","""DisWTGDEh03""","""C05""",35.0,"""FVrry_P05_Q01""","""D01""",4,"""G01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",5613.0,null,null,null,null,null,null,null,"""B15A1A2P1-G01"""
"""B15A1A2P1_R1""","""I01""","""APOA1""","""APOA1_Arg34Leu""","""VUSmut_GDEh09""","""C12""",91.0,"""FVrry_P05_Q01""","""E01""",5,"""I01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",5613.0,"""101G>T""","""Arg34Leu""",215445.0,null,null,null,null,"""B15A1A2P1-I01"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""B15A1A2P1_R1""","""D08""","""TPM1""","""TPM1_Glu40Lys""","""VUSmut_GDEh12""","""G11""",87.0,"""FVrry_P05_Q04""","""B04""",26,"""D08""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4723.0,"""118G>A""","""Glu40Lys""",8945.0,null,null,null,null,"""B15A1A2P1-D08"""
"""B15A1A2P1_R1""","""F08""","""TPM1""","""TPM1_Val95Ala""","""VUSmut_GDEh12""","""H11""",88.0,"""FVrry_P05_Q04""","""C04""",27,"""F08""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4723.0,"""284T>C""","""Val95Ala""",8950.0,null,null,null,null,"""B15A1A2P1-F08"""
"""B15A1A2P1_R1""","""J08""","""TPM3""","""TPM3""","""DisWTGDEh03""","""F07""",54.0,"""FVrry_P05_Q04""","""E04""",29,"""J08""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",5712.0,null,null,null,null,null,null,null,"""B15A1A2P1-J08"""


plate,well,channel,perc_25,perc_50,perc_75,perc_80,perc_90,perc_95,perc_99,mean,std,median_plate,mean_plate,std_plate,s2n_ratio,s2n_threshold,is_bg,plate_id,plate_well,plate_map_name,well_position,symbol,gene_allele,source_plate,source_well,source_numb,dest_plate,dest_well,dest_numb,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,nt_change_cdna,aa_change,ccsb_mutation_id,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,sequencing_confidence
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,bool,str,str,str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,str,str,f64,str,str,str,str
"""2025-03-17_B15A1A2_P1T4""","""P19""","""GFP""",131.0,145.0,157.0,160.0,167.0,172.0,181.0,143.814015,19.423083,153,439.154452,2664.626457,0.140439,0.186343,true,"""B15A1A2P1""","""B15A1A2P1-P19""","""B15A1A2P1_R1""","""P19""","""SMAD3""","""SMAD3_Arg279Lys""","""VUSmut_GDEh07""","""G04""",31.0,"""FVrry_P05_Q03""","""H10""",80,"""P19""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",11805.0,"""836G>A""","""Arg279Lys""",203608.0,null,null,null,null
"""2025-03-17_B15A1A2_P1T4""","""D07""","""GFP""",129.0,143.0,156.0,158.0,165.0,170.0,179.0,141.881527,18.377427,153,439.154452,2664.626457,0.142295,0.186343,true,"""B15A1A2P1""","""B15A1A2P1-D07""","""B15A1A2P1_R1""","""D07""","""PNKP""","""PNKP_Pro20Ser""","""VUSmut_GDEh06""","""F04""",30.0,"""FVrry_P05_Q03""","""B04""",26,"""D07""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",9905.0,"""58C>T""","""Pro20Ser""",216950.0,null,null,null,null
"""2025-03-17_B15A1A2_P1T4""","""J01""","""GFP""",126.0,140.0,152.0,155.0,161.0,166.0,175.0,138.755857,18.375394,153,439.154452,2664.626457,0.142699,0.186343,true,"""B15A1A2P1""","""B15A1A2P1-J01""","""B15A1A2P1_R1""","""J01""","""NR0B1""","""NR0B1_Lys382Asn""","""VUSmut_GDEh10""","""A04""",25.0,"""FVrry_P05_Q03""","""E01""",5,"""J01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",7215.0,"""1146G>T""","""Lys382Asn""",216199.0,null,null,null,null
"""2025-03-17_B15A1A2_P1T4""","""P06""","""GFP""",128.0,142.0,155.0,157.0,164.0,169.0,178.0,141.257999,19.625451,153,439.154452,2664.626457,0.143241,0.186343,true,"""B15A1A2P1""","""B15A1A2P1-P06""","""B15A1A2P1_R1""","""P06""","""TPM1""","""TPM1""","""FV_reLR_02""","""B01""",2.0,"""FVrry_P05_Q04""","""H03""",24,"""P06""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",4723.0,"""523G>A""","""Asp175Asn""",8952.0,null,null,null,"""verified """
"""2025-03-17_B15A1A2_P1T4""","""J19""","""GFP""",130.0,145.0,157.0,160.0,166.0,171.0,181.0,143.454732,30.135292,153,439.154452,2664.626457,0.143766,0.186343,true,"""B15A1A2P1""","""B15A1A2P1-J19""","""B15A1A2P1_R1""","""J19""","""SMAD3""","""SMAD3_Glu239Lys""","""VUSmut_GDEh06""","""E12""",93.0,"""FVrry_P05_Q03""","""E10""",77,"""J19""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",11805.0,"""715G>A""","""Glu239Lys""",203609.0,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2025-03-17_B15A1A2_P1T3""","""C19""","""GFP""",147.0,169.0,336.0,574.0,1607.0,3703.0,60339.0,1627.117934,7648.379636,153,445.507364,2511.020108,2.613281,0.185893,false,"""B15A1A2P1""","""B15A1A2P1-C19""","""B15A1A2P1_R1""","""C19""","""GFAP""","""GFAP_Arg79Ser""","""VUSmut_GDEh08""","""A03""",17.0,"""FVrry_P05_Q01""","""B10""",74,"""C19""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",2920.0,"""235C>A""","""Arg79Ser""",214406.0,null,null,null,null
"""2025-03-17_B15A1A2_P1T3""","""I21""","""GFP""",146.0,169.0,421.0,664.0,1489.0,2956.0,60770.0,1604.456372,7685.735556,153,445.507364,2511.020108,2.619336,0.185893,false,"""B15A1A2P1""","""B15A1A2P1-I21""","""B15A1A2P1_R1""","""I21""","""GFAP""","""GFAP_Tyr366Cys""","""VUSmut_GDEh08""","""H11""",88.0,"""FVrry_P05_Q01""","""E11""",85,"""I21""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",2920.0,"""1097A>G""","""Tyr366Cys""",214440.0,null,null,null,null
"""2025-03-17_B15A1A2_P1T3""","""E21""","""GFP""",144.0,165.0,271.0,579.0,1638.

gene_allele,well,is_bg_sum,Batch,QC_Flag
str,str,u32,str,str
"""KCNJ2_Met301Leu""","""G06""",0,"""2025_03_17_Batch_15""","""PASS"""
"""SFTPA2_Gln223Lys""","""P07""",0,"""2025_03_17_Batch_15""","""PASS"""
"""SMAD3_Leu274Pro""","""P15""",0,"""2025_03_17_Batch_15""","""PASS"""
"""TPM1_Lys37Glu""","""F02""",0,"""2025_03_17_Batch_15""","""PASS"""
"""LITAF_Trp116Gly""","""G24""",0,"""2025_03_17_Batch_15""","""PASS"""
…,…,…,…,…
"""TMEM43_Ala366Thr""","""N23""",0,"""2025_03_17_Batch_15""","""PASS"""
"""SMAD3_Ser355Leu""","""L17""",0,"""2025_03_17_Batch_15""","""PASS"""
"""LITAF_Thr115Asn""","""M18""",0,"""2025_03_17_Batch_15""","""PASS"""


plate,well,channel,perc_25,perc_50,perc_75,perc_80,perc_90,perc_95,perc_99,mean,std,median_plate,mean_plate,std_plate,s2n_ratio,s2n_threshold,is_bg,plate_id,plate_well,plate_map_name,well_position,symbol,gene_allele,source_plate,source_well,source_numb,dest_plate,dest_well,dest_numb,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,nt_change_cdna,aa_change,ccsb_mutation_id,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,sequencing_confidence
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,bool,str,str,str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,str,str,f64,str,str,str,str
"""2025-03-17_B15A1A2_P1T4""","""D07""","""DAPI""",161.0,180.0,197.0,201.0,209.0,216.0,226.0,178.472598,24.98599,180,188.175043,68.561838,1.403727,1.454161,true,"""B15A1A2P1""","""B15A1A2P1-D07""","""B15A1A2P1_R1""","""D07""","""PNKP""","""PNKP_Pro20Ser""","""VUSmut_GDEh06""","""F04""",30.0,"""FVrry_P05_Q03""","""B04""",26,"""D07""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",9905.0,"""58C>T""","""Pro20Ser""",216950.0,null,null,null,null
"""2025-03-17_B15A1A2_P1T4""","""J01""","""DAPI""",152.0,170.0,186.0,190.0,198.0,204.0,215.0,169.422638,29.294912,180,188.175043,68.561838,1.414474,1.454161,true,"""B15A1A2P1""","""B15A1A2P1-J01""","""B15A1A2P1_R1""","""J01""","""NR0B1""","""NR0B1_Lys382Asn""","""VUSmut_GDEh10""","""A04""",25.0,"""FVrry_P05_Q03""","""E01""",5,"""J01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",7215.0,"""1146G>T""","""Lys382Asn""",216199.0,null,null,null,null
"""2025-03-17_B15A1A2_P1T4""","""P06""","""DAPI""",159.0,178.0,195.0,199.0,207.0,214.0,226.0,177.408812,35.598683,180,188.175043,68.561838,1.421384,1.454161,true,"""B15A1A2P1""","""B15A1A2P1-P06""","""B15A1A2P1_R1""","""P06""","""TPM1""","""TPM1""","""FV_reLR_02""","""B01""",2.0,"""FVrry_P05_Q04""","""H03""",24,"""P06""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",4723.0,"""523G>A""","""Asp175Asn""",8952.0,null,null,null,"""verified """
"""2025-03-17_B15A1A2_P1T4""","""J19""","""DAPI""",158.0,178.0,195.0,199.0,207.0,214.0,225.0,176.159846,29.256178,180,188.175043,68.561838,1.424051,1.454161,true,"""B15A1A2P1""","""B15A1A2P1-J19""","""B15A1A2P1_R1""","""J19""","""SMAD3""","""SMAD3_Glu239Lys""","""VUSmut_GDEh06""","""E12""",93.0,"""FVrry_P05_Q03""","""E10""",77,"""J19""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",11805.0,"""715G>A""","""Glu239Lys""",203609.0,null,null,null,null
"""2025-03-17_B15A1A2_P1T4""","""J05""","""DAPI""",157.0,176.0,193.0,197.0,206.0,212.0,224.0,175.061106,29.156054,180,188.175043,68.561838,1.426752,1.454161,true,"""B15A1A2P1""","""B15A1A2P1-J05""","""B15A1A2P1_R1""","""J05""","""PHYH""","""PHYH_Arg275Gln""","""VUSmut_GDEh06""","""C03""",19.0,"""FVrry_P05_Q03""","""E03""",21,"""J05""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",2156.0,"""824G>A""","""Arg275Gln""",214170.0,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2025-03-17_B16A1A2_P1T1""","""M21""","""Mito""",520.0,643.0,1292.0,1575.0,2490.0,3577.0,10039.0,1265.02386,1984.914957,580,908.000392,1030.850869,19.305769,1.941429,false,"""B16A1A2P1""","""B16A1A2P1-M21""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2025-03-17_B16A1A2_P1T1""","""M19""","""Mito""",530.0,653.0,1031.0,1310.0,2295.0,3448.0,10624.0,1208.298958,2080.264216,580,908.000392,1030.850869,20.045283,1.941429,false,"""B16A1A2P1""","""B16A1A2P1-M19""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2025-03-17_B16A1A2_P1T1""","""E21""","""Mito""",599.0,751.0,1282.0,1614.0,2706.0,4076.0,12462.0,1418.562383,2334.758638,580,908.000392,1030.850869,20.804674,1.941429,false,"""B16A1A2P1""","""B16A1A2P1-E21""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


plate_map_name,well_position,symbol,gene_allele,source_plate,source_well,source_numb,dest_plate,dest_well,dest_numb,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,nt_change_cdna,aa_change,ccsb_mutation_id,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,sequencing_confidence,plate_well
str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,str,str,f64,str,str,str,str,str
"""B16A1A2P1_R1""","""A01""","""AP2S1""","""AP2S1""","""FV_reLR_01""","""G01""",7.0,"""FVrry_P05_Q01""","""A01""",1,"""A01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",4983.0,null,null,null,null,null,null,"""high ""","""B16A1A2P1-A01"""
"""B16A1A2P1_R1""","""C01""","""AP2S1""","""AP2S1_Arg15Cys""","""VUSmut_GDEh06""","""B04""",26.0,"""FVrry_P05_Q01""","""B01""",2,"""C01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4983.0,"""43C>T""","""Arg15Cys""",215314.0,null,null,null,null,"""B16A1A2P1-C01"""
"""B16A1A2P1_R1""","""E01""","""AP2S1""","""AP2S1_Arg15His""","""VUSmut_GDEh06""","""G05""",39.0,"""FVrry_P05_Q01""","""C01""",3,"""E01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4983.0,"""44G>A""","""Arg15His""",215316.0,null,null,null,null,"""B16A1A2P1-E01"""
"""B16A1A2P1_R1""","""G01""","""APOA1""","""APOA1""","""DisWTGDEh03""","""C05""",35.0,"""FVrry_P05_Q01""","""D01""",4,"""G01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",5613.0,null,null,null,null,null,null,null,"""B16A1A2P1-G01"""
"""B16A1A2P1_R1""","""I01""","""APOA1""","""APOA1_Arg34Leu""","""VUSmut_GDEh09""","""C12""",91.0,"""FVrry_P05_Q01""","""E01""",5,"""I01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",5613.0,"""101G>T""","""Arg34Leu""",215445.0,null,null,null,null,"""B16A1A2P1-I01"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""B16A1A2P1_R1""","""D08""","""TPM1""","""TPM1_Glu40Lys""","""VUSmut_GDEh12""","""G11""",87.0,"""FVrry_P05_Q04""","""B04""",26,"""D08""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4723.0,"""118G>A""","""Glu40Lys""",8945.0,null,null,null,null,"""B16A1A2P1-D08"""
"""B16A1A2P1_R1""","""F08""","""TPM1""","""TPM1_Val95Ala""","""VUSmut_GDEh12""","""H11""",88.0,"""FVrry_P05_Q04""","""C04""",27,"""F08""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4723.0,"""284T>C""","""Val95Ala""",8950.0,null,null,null,null,"""B16A1A2P1-F08"""
"""B16A1A2P1_R1""","""J08""","""TPM3""","""TPM3""","""DisWTGDEh03""","""F07""",54.0,"""FVrry_P05_Q04""","""E04""",29,"""J08""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",5712.0,null,null,null,null,null,null,null,"""B16A1A2P1-J08"""


plate,well,channel,perc_25,perc_50,perc_75,perc_80,perc_90,perc_95,perc_99,mean,std,median_plate,mean_plate,std_plate,s2n_ratio,s2n_threshold,is_bg,plate_id,plate_well,plate_map_name,well_position,symbol,gene_allele,source_plate,source_well,source_numb,dest_plate,dest_well,dest_numb,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,nt_change_cdna,aa_change,ccsb_mutation_id,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,sequencing_confidence,plate_map_name_right,well_position_right,symbol_right,gene_allele_right,source_plate_right,source_well_right,source_numb_right,dest_plate_right,dest_well_right,dest_numb_right,imaging_well_right,imaging_plate_R1_right,imaging_plate_R2_right,node_type_right,orf_id_wt_right,nt_change_cdna_right,aa_change_right,ccsb_mutation_id_right,hgmd_ID_right,clinvar_allele_ID_right,clinvar_clinical_significance_right,sequencing_confidence_right
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,bool,str,str,str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,str,str,f64,str,str,str,str,str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,str,str,f64,str,str,str,str
"""2025-03-17_B15A1A2_P1T4""","""P19""","""GFP""",131.0,145.0,157.0,160.0,167.0,172.0,181.0,143.814015,19.423083,153,439.154452,2664.626457,0.140439,0.186343,true,"""B15A1A2P1""","""B15A1A2P1-P19""","""B15A1A2P1_R1""","""P19""","""SMAD3""","""SMAD3_Arg279Lys""","""VUSmut_GDEh07""","""G04""",31.0,"""FVrry_P05_Q03""","""H10""",80,"""P19""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",11805.0,"""836G>A""","""Arg279Lys""",203608.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2025-03-17_B15A1A2_P1T4""","""D07""","""GFP""",129.0,143.0,156.0,158.0,165.0,170.0,179.0,141.881527,18.377427,153,439.154452,2664.626457,0.142295,0.186343,true,"""B15A1A2P1""","""B15A1A2P1-D07""","""B15A1A2P1_R1""","""D07""","""PNKP""","""PNKP_Pro20Ser""","""VUSmut_GDEh06""","""F04""",30.0,"""FVrry_P05_Q03""","""B04""",26,"""D07""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",9905.0,"""58C>T""","""Pro20Ser""",216950.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2025-03-17_B15A1A2_P1T4""","""J01""","""GFP""",126.0,140.0,152.0,155.0,161.0,166.0,175.0,138.755857,18.375394,153,439.154452,2664.626457,0.142699,0.186343,true,"""B15A1A2P1""","""B15A1A2P1-J01""","""B15A1A2P1_R1""","""J01""","""NR0B1""","""NR0B1_Lys382Asn""","""VUSmut_GDEh10""","""A04""",25.0,"""FVrry_P05_Q03""","""E01""",5,"""J01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",7215.0,"""1146G>T""","""Lys382Asn""",216199.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2025-03-17_B15A1A2_P1T4""","""P06""","""GFP""",128.0,142.0,155.0,157.0,164.0,169.0,178.0,141.257999,19.625451,153,439.154452,2664.626457,0.143241,0.186343,true,"""B15A1A2P1""","""B15A1A2P1-P06""","""B15A1A2P1_R1""","""P06""","""TPM1""","""TPM1""","""FV_reLR_02""","""B01""",2.0,"""FVrry_P05_Q04""","""H03""",24,"""P06""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",4723.0,"""523G>A""","""Asp175Asn""",8952.0,null,null,null,"""verified """,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2025-03-17_B15A1A2_P1T4""","""J19""","""GFP""",130.0,145.0,157.0,160.0,166.0,171.0,181.0,143.454732,30.135292,153,439.154452,2664.626457,0.143766,0.186343,true,"""B15A1A2P1""","""B15A1A2P1-J19""","""B15A1A2P1_R1""","""J19""","""SMAD3""","""SMAD3_Glu239Lys""","""VUSmut_GDEh06""","""E12""",93.0,"""FVrry_P05_Q03""","""E10""",77,"""J19""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",11805.0,"""715G>A""","""Glu239Lys""",203609.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…

gene_allele,well,is_bg_sum,Batch,QC_Flag
str,str,u32,str,str
"""IL36RN_Leu27Pro""","""I02""",0,"""2025_03_17_Batch_16""","""PASS"""
"""KCNJ2_Val93Ile""","""A06""",0,"""2025_03_17_Batch_16""","""PASS"""
"""SMAD3_Leu274Pro""","""P15""",0,"""2025_03_17_Batch_16""","""PASS"""
"""APOA1""","""G01""",0,"""2025_03_17_Batch_16""","""PASS"""
"""GFAP_Arg239Leu""","""G21""",0,"""2025_03_17_Batch_16""","""PASS"""
…,…,…,…,…
"""KCNJ2_Ala115Val""","""K08""",2,"""2025_03_17_Batch_16""","""PASS"""
"""TMEM43_Glu85Lys""","""F23""",1,"""2025_03_17_Batch_16""","""PASS"""
"""LITAF_Pro84Leu""","""A22""",0,"""2025_03_17_Batch_16""","""PASS"""


## Batch 11-12

In [111]:
BIO_REP_BATCHES = ["2024_12_09_Batch_11", "2024_12_09_Batch_12"]
COMBINED_BIO_REP_DIR = "2024_12_Batch_11-12"

OUTPUT_DIR = f"../../outputs/{COMBINED_BIO_REP_DIR}"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [112]:
allele_meta_dict = {}
for batch_id in BIO_REP_BATCHES:
    allele_meta_df = pl.DataFrame()
    for platemap in os.listdir(PLATEMAP_DIR.format(batch_id=batch_id)):
        platemap_df = pl.read_csv(os.path.join(PLATEMAP_DIR.format(batch_id=batch_id), platemap), separator="\t", infer_schema_length=100000)
        allele_meta_df = pl.concat([allele_meta_df, 
                                    platemap_df.filter((~pl.col("node_type").is_in(["TC","NC","PC"]))&(pl.col("node_type").is_not_null()))], 
                                    how="diagonal_relaxed").sort("plate_map_name")
        allele_meta_df = allele_meta_df.with_columns(pl.col("plate_map_name").str.split('_').list.get(0).alias("plate_map"))
    allele_meta_dict[batch_id] = allele_meta_df
    display(allele_meta_df)#.sort()

plate_map_name,well_position,symbol,gene_allele,source_plate,source_well,source_numb,dest_plate,dest_well,dest_numb,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,technical_replicate,batch1,batch2,clone_ID,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,vector,notes,plate,quad,c96,r96,r384,c384,r384_letter,gene_symbol,symbol_original,plate_map
str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,f64,str,str,f64,i64,i64,i64,i64,str,str,str,str,i64,str,i64,i64,i64,i64,i64,i64,str,str,str,str
"""B11A1R1_P1""","""A01""","""MLH1""","""MLH1""","""Tube""",null,null,"""1%_rescreen_P01_R1""","""A01""",1,"""A01""","""B11A1R1""","""B12A1R2""","""disease_wt""",2713.0,null,null,null,1.0,1,1,11,12,null,null,null,null,527,null,1,1,1,1,1,1,"""A""","""MLH1""","""MLH1""","""B11A1R1"""
"""B11A1R1_P1""","""C01""","""MLH1""","""MLH1_Ala29Gly""","""VUSmut_GDEh03""","""H11""",88.0,"""1%_rescreen_P01_R1""","""B01""",2,"""C01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,451.0,"""86C>G""","""Ala29Gly""",1.0,1,1,11,12,null,null,null,null,527,null,1,1,1,2,3,1,"""C""","""MLH1""","""MLH1""","""B11A1R1"""
"""B11A1R1_P1""","""E01""","""MLH1""","""MLH1_Asn38His""","""VUSmut_GDEh03""","""D12""",92.0,"""1%_rescreen_P01_R1""","""C01""",3,"""E01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,467.0,"""112A>C""","""Asn38His""",1.0,1,1,11,12,null,null,null,null,527,null,1,1,1,3,5,1,"""E""","""MLH1""","""MLH1""","""B11A1R1"""
"""B11A1R1_P1""","""G01""","""MLH1""","""MLH1_Asn38Lys""","""VUSmut_GDEh03""","""E12""",93.0,"""1%_rescreen_P01_R1""","""D01""",4,"""G01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,470.0,"""114C>G""","""Asn38Lys""",1.0,1,1,11,12,null,null,null,null,527,null,1,1,1,4,7,1,"""G""","""MLH1""","""MLH1""","""B11A1R1"""
"""B11A1R1_P1""","""I01""","""MLH1""","""MLH1_Cys39Tyr""","""VUSmut_GDEh03""","""F12""",94.0,"""1%_rescreen_P01_R1""","""E01""",5,"""I01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,472.0,"""116G>A""","""Cys39Tyr""",1.0,1,1,11,12,null,null,null,null,527,null,1,1,1,5,9,1,"""I""","""MLH1""","""MLH1""","""B11A1R1"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""B11A1R1_P1""","""L22""","""AGXT""","""AGXT_Asp201Asn""","""594 Controls """,null,null,"""1%_rescreen_P01_R1""","""F11""",86,"""L22""","""B11A1R1""","""B12A1R2""","""cPC""",null,null,null,null,null,1,4,11,12,null,null,null,null,594,null,1,4,11,6,12,22,"""L""","""AGXT""","""AGXT""","""B11A1R1"""
"""B11A1R1_P1""","""N22""","""GSS""","""GSS""","""594 Controls """,null,null,"""1%_rescreen_P01_R1""","""G11""",87,"""N22""","""B11A1R1""","""B12A1R2""","""cPC""",null,null,null,null,null,1,4,11,12,null,null,null,null,594,null,1,4,11,7,14,22,"""N""","""GSS""","""GSS""","""B11A1R1"""
"""B11A1R1_P1""","""P22""","""GSS""","""GSS_Arg125Cys""","""594 Controls """,null,null,"""1%_rescreen_P01_R1""","""H11""",88,"""P22""","""B11A1R1""","""B12A1R2""","""cPC""",null,null,null,null,null,1,4,11,12,null,null,null,null,594,null,1,4,11,8,16,22,"""P""","""GSS""","""GSS""","""B11A1R1"""


plate_map_name,well_position,symbol,gene_allele,source_plate,source_well,source_numb,dest_plate,dest_well,dest_numb,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,technical_replicate,batch1,batch2,clone_ID,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,vector,notes,plate,quad,c96,r96,r384,c384,r384_letter,gene_symbol,symbol_original,plate_map
str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,f64,str,str,f64,i64,i64,i64,i64,str,str,str,str,i64,str,i64,i64,i64,i64,i64,i64,str,str,str,str
"""B12A1R2_P1""","""A01""","""MLH1""","""MLH1""","""Tube""",null,null,"""1%_rescreen_P01_R1""","""A01""",1,"""A01""","""B11A1R1""","""B12A1R2""","""disease_wt""",2713.0,null,null,null,1.0,1,1,11,12,null,null,null,null,527,null,1,1,1,1,1,1,"""A""","""MLH1""","""MLH1""","""B12A1R2"""
"""B12A1R2_P1""","""C01""","""MLH1""","""MLH1_Ala29Gly""","""VUSmut_GDEh03""","""H11""",88.0,"""1%_rescreen_P01_R1""","""B01""",2,"""C01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,451.0,"""86C>G""","""Ala29Gly""",1.0,1,1,11,12,null,null,null,null,527,null,1,1,1,2,3,1,"""C""","""MLH1""","""MLH1""","""B12A1R2"""
"""B12A1R2_P1""","""E01""","""MLH1""","""MLH1_Asn38His""","""VUSmut_GDEh03""","""D12""",92.0,"""1%_rescreen_P01_R1""","""C01""",3,"""E01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,467.0,"""112A>C""","""Asn38His""",1.0,1,1,11,12,null,null,null,null,527,null,1,1,1,3,5,1,"""E""","""MLH1""","""MLH1""","""B12A1R2"""
"""B12A1R2_P1""","""G01""","""MLH1""","""MLH1_Asn38Lys""","""VUSmut_GDEh03""","""E12""",93.0,"""1%_rescreen_P01_R1""","""D01""",4,"""G01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,470.0,"""114C>G""","""Asn38Lys""",1.0,1,1,11,12,null,null,null,null,527,null,1,1,1,4,7,1,"""G""","""MLH1""","""MLH1""","""B12A1R2"""
"""B12A1R2_P1""","""I01""","""MLH1""","""MLH1_Cys39Tyr""","""VUSmut_GDEh03""","""F12""",94.0,"""1%_rescreen_P01_R1""","""E01""",5,"""I01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,472.0,"""116G>A""","""Cys39Tyr""",1.0,1,1,11,12,null,null,null,null,527,null,1,1,1,5,9,1,"""I""","""MLH1""","""MLH1""","""B12A1R2"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""B12A1R2_P1""","""L22""","""AGXT""","""AGXT_Asp201Asn""","""594 Controls """,null,null,"""1%_rescreen_P01_R1""","""F11""",86,"""L22""","""B11A1R1""","""B12A1R2""","""cPC""",null,null,null,null,null,1,4,11,12,null,null,null,null,594,null,1,4,11,6,12,22,"""L""","""AGXT""","""AGXT""","""B12A1R2"""
"""B12A1R2_P1""","""N22""","""GSS""","""GSS""","""594 Controls """,null,null,"""1%_rescreen_P01_R1""","""G11""",87,"""N22""","""B11A1R1""","""B12A1R2""","""cPC""",null,null,null,null,null,1,4,11,12,null,null,null,null,594,null,1,4,11,7,14,22,"""N""","""GSS""","""GSS""","""B12A1R2"""
"""B12A1R2_P1""","""P22""","""GSS""","""GSS_Arg125Cys""","""594 Controls """,null,null,"""1%_rescreen_P01_R1""","""H11""",88,"""P22""","""B11A1R1""","""B12A1R2""","""cPC""",null,null,null,null,null,1,4,11,12,null,null,null,null,594,null,1,4,11,8,16,22,"""P""","""GSS""","""GSS""","""B12A1R2"""


In [113]:
plate_img_qc = pl.DataFrame()
well_img_qc_gfp_sum = pl.DataFrame()

for batch_id in BIO_REP_BATCHES:
    plate_img_qc = pl.concat([plate_img_qc, pl.read_parquet(f"{IMGS_QC_DIR}/{batch_id}/plate_well_sum_with_metrics.parquet")], how="diagonal_relaxed")
    plate_img_qc = plate_img_qc.with_columns(
        pl.col("plate").str.split("_").list.get(-1).alias("plate_id")
    ).with_columns(
        pl.concat_str([pl.col("plate_id"),pl.col("well")], separator="-").alias("plate_well")
    )#.filter((pl.col("plate").str.contains("B7A"))&(pl.col("channel")=="GFP"))
    display(plate_img_qc)

    allele_meta_df = allele_meta_dict[batch_id].with_columns(
        pl.concat_str([pl.col("plate_map"), pl.col("well_position")], separator="-").alias("plate_well")
    ).drop("plate_map","plate")
    display(allele_meta_df)
    plate_img_qc = plate_img_qc.join(
        allele_meta_df, 
        on="plate_well", how="left"
    ).filter(
        (pl.col("gene_allele").is_not_null())
    )
    # display(plate_img_qc)
    plate_img_qc_gfp = plate_img_qc.filter(pl.col("channel")=="GFP")
    display(plate_img_qc_gfp)
    
    well_img_qc_gfp_sum_batch = plate_img_qc_gfp.group_by(["gene_allele","well"]).agg(
        pl.col("is_bg").sum().alias("is_bg_sum"),
    ).with_columns(
        pl.lit(batch_id).alias("Batch"),
        pl.when(pl.col("is_bg_sum")>=3)
        .then(pl.lit("FAIL"))
        .otherwise(pl.lit("PASS"))
        .alias("QC_Flag")
    )
    display(well_img_qc_gfp_sum_batch)

    well_img_qc_gfp_sum = pl.concat([well_img_qc_gfp_sum, well_img_qc_gfp_sum_batch])

well_img_qc_gfp_sum.write_csv(f"{OUTPUT_DIR}/well-level_img_qc_gfp_sum.csv")

plate,well,channel,mean,std,perc_25,perc_50,perc_75,perc_80,perc_90,perc_95,perc_99,median_plate,mean_plate,std_plate,s2n_ratio,s2n_threshold,is_bg,plate_id,plate_well
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,bool,str,str
"""2024-12-09_B11A1R1""","""M24""","""DAPI""",184.65211,117.898848,163.0,183.0,201.0,205.0,213.0,220.0,234.0,198,286.685087,382.698537,1.435583,1.652741,true,"""B11A1R1""","""B11A1R1-M24"""
"""2024-12-09_B11A1R1""","""L11""","""DAPI""",197.793606,116.244789,175.0,197.0,217.0,222.0,231.0,239.0,252.0,198,286.685087,382.698537,1.44,1.652741,true,"""B11A1R1""","""B11A1R1-L11"""
"""2024-12-09_B11A1R1""","""L12""","""DAPI""",196.757655,68.624404,175.0,197.0,217.0,221.0,231.0,238.0,253.0,198,286.685087,382.698537,1.445714,1.652741,true,"""B11A1R1""","""B11A1R1-L12"""
"""2024-12-09_B11A1R1""","""E06""","""DAPI""",189.426109,74.603005,168.0,190.0,209.0,213.0,222.0,229.0,243.0,198,286.685087,382.698537,1.446429,1.652741,true,"""B11A1R1""","""B11A1R1-E06"""
"""2024-12-09_B11A1R1""","""H11""","""DAPI""",193.331611,141.487152,171.0,193.0,213.0,217.0,227.0,235.0,248.0,198,286.685087,382.698537,1.450292,1.652741,true,"""B11A1R1""","""B11A1R1-H11"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2024-12-09_B11A1R1_widefield""","""J22""","""Mito""",2461.818816,2365.612645,1313.0,1527.0,2424.0,3060.0,5333.0,7349.0,12103.0,2179,3123.945321,2550.906126,9.217822,1.615775,false,"""widefield""","""widefield-J22"""
"""2024-12-09_B11A1R1_widefield""","""G02""","""Mito""",3387.746931,3072.203043,1583.0,1934.0,4278.0,5029.0,7216.0,9400.0,14725.0,2179,3123.945321,2550.906126,9.301958,1.615775,false,"""widefield""","""widefield-G02"""
"""2024-12-09_B11A1R1_widefield""","""G22""","""Mito""",3204.770072,3251.200154,1671.0,1955.0,3251.0,4057.0,6852.0,9419.0,15562.0,2179,3123.945321,2550.906126,9.312986,1.615775,false,"""widefield""","""widefield-G22"""


plate_map_name,well_position,symbol,gene_allele,source_plate,source_well,source_numb,dest_plate,dest_well,dest_numb,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,technical_replicate,batch1,batch2,clone_ID,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,vector,notes,quad,c96,r96,r384,c384,r384_letter,gene_symbol,symbol_original,plate_well
str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,f64,str,str,f64,i64,i64,i64,i64,str,str,str,str,i64,str,i64,i64,i64,i64,i64,str,str,str,str
"""B11A1R1_P1""","""A01""","""MLH1""","""MLH1""","""Tube""",null,null,"""1%_rescreen_P01_R1""","""A01""",1,"""A01""","""B11A1R1""","""B12A1R2""","""disease_wt""",2713.0,null,null,null,1.0,1,1,11,12,null,null,null,null,527,null,1,1,1,1,1,"""A""","""MLH1""","""MLH1""","""B11A1R1-A01"""
"""B11A1R1_P1""","""C01""","""MLH1""","""MLH1_Ala29Gly""","""VUSmut_GDEh03""","""H11""",88.0,"""1%_rescreen_P01_R1""","""B01""",2,"""C01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,451.0,"""86C>G""","""Ala29Gly""",1.0,1,1,11,12,null,null,null,null,527,null,1,1,2,3,1,"""C""","""MLH1""","""MLH1""","""B11A1R1-C01"""
"""B11A1R1_P1""","""E01""","""MLH1""","""MLH1_Asn38His""","""VUSmut_GDEh03""","""D12""",92.0,"""1%_rescreen_P01_R1""","""C01""",3,"""E01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,467.0,"""112A>C""","""Asn38His""",1.0,1,1,11,12,null,null,null,null,527,null,1,1,3,5,1,"""E""","""MLH1""","""MLH1""","""B11A1R1-E01"""
"""B11A1R1_P1""","""G01""","""MLH1""","""MLH1_Asn38Lys""","""VUSmut_GDEh03""","""E12""",93.0,"""1%_rescreen_P01_R1""","""D01""",4,"""G01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,470.0,"""114C>G""","""Asn38Lys""",1.0,1,1,11,12,null,null,null,null,527,null,1,1,4,7,1,"""G""","""MLH1""","""MLH1""","""B11A1R1-G01"""
"""B11A1R1_P1""","""I01""","""MLH1""","""MLH1_Cys39Tyr""","""VUSmut_GDEh03""","""F12""",94.0,"""1%_rescreen_P01_R1""","""E01""",5,"""I01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,472.0,"""116G>A""","""Cys39Tyr""",1.0,1,1,11,12,null,null,null,null,527,null,1,1,5,9,1,"""I""","""MLH1""","""MLH1""","""B11A1R1-I01"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""B11A1R1_P1""","""L22""","""AGXT""","""AGXT_Asp201Asn""","""594 Controls """,null,null,"""1%_rescreen_P01_R1""","""F11""",86,"""L22""","""B11A1R1""","""B12A1R2""","""cPC""",null,null,null,null,null,1,4,11,12,null,null,null,null,594,null,4,11,6,12,22,"""L""","""AGXT""","""AGXT""","""B11A1R1-L22"""
"""B11A1R1_P1""","""N22""","""GSS""","""GSS""","""594 Controls """,null,null,"""1%_rescreen_P01_R1""","""G11""",87,"""N22""","""B11A1R1""","""B12A1R2""","""cPC""",null,null,null,null,null,1,4,11,12,null,null,null,null,594,null,4,11,7,14,22,"""N""","""GSS""","""GSS""","""B11A1R1-N22"""
"""B11A1R1_P1""","""P22""","""GSS""","""GSS_Arg125Cys""","""594 Controls """,null,null,"""1%_rescreen_P01_R1""","""H11""",88,"""P22""","""B11A1R1""","""B12A1R2""","""cPC""",null,null,null,null,null,1,4,11,12,null,null,null,null,594,null,4,11,8,16,22,"""P""","""GSS""","""GSS""","""B11A1R1-P22"""


plate,well,channel,mean,std,perc_25,perc_50,perc_75,perc_80,perc_90,perc_95,perc_99,median_plate,mean_plate,std_plate,s2n_ratio,s2n_threshold,is_bg,plate_id,plate_well,plate_map_name,well_position,symbol,gene_allele,source_plate,source_well,source_numb,dest_plate,dest_well,dest_numb,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,technical_replicate,batch1,batch2,clone_ID,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,vector,notes,quad,c96,r96,r384,c384,r384_letter,gene_symbol,symbol_original
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,bool,str,str,str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,f64,str,str,f64,i64,i64,i64,i64,str,str,str,str,i64,str,i64,i64,i64,i64,i64,str,str,str
"""2024-12-09_B11A1R1""","""G12""","""GFP""",154.660478,33.916139,141.0,156.0,169.0,172.0,179.0,184.0,193.0,178,896.607129,3527.121546,0.13637,0.180849,true,"""B11A1R1""","""B11A1R1-G12""","""B11A1R1_P1""","""G12""","""MVK""","""MVK_IIe268Thr""",null,null,null,"""1%_rescreen_P01_R1""","""D06""",44,"""G12""","""B11A1R1""","""B12A1R2""","""allele""",281.0,20.0,"""803T>C""","""Ile268Thr""",1.0,1,2,11,12,null,null,null,null,527,null,2,6,4,7,12,"""G""","""MVK""","""MVK"""
"""2024-12-09_B11A1R1""","""H12""","""GFP""",155.102661,30.689846,141.0,156.0,169.0,172.0,179.0,184.0,193.0,178,896.607129,3527.121546,0.13637,0.180849,true,"""B11A1R1""","""B11A1R1-H12""","""B11A1R1_P1""","""H12""","""MVK""","""MVK_IIe268Thr""",null,null,null,"""1%_rescreen_P01_R1""","""D06""",44,"""H12""","""B11A1R1""","""B12A1R2""","""allele""",281.0,20.0,"""803T>C""","""Ile268Thr""",1.0,1,4,11,12,null,null,null,null,527,null,4,6,4,8,12,"""H""","""MVK""","""MVK"""
"""2024-12-09_B11A1R1""","""H11""","""GFP""",154.026189,19.641371,140.0,155.0,169.0,171.0,178.0,183.0,192.0,178,896.607129,3527.121546,0.137205,0.180849,true,"""B11A1R1""","""B11A1R1-H11""","""B11A1R1_P1""","""H11""","""MVK""","""MVK_IIe268Thr""",null,null,null,"""1%_rescreen_P01_R1""","""D06""",44,"""H11""","""B11A1R1""","""B12A1R2""","""allele""",281.0,20.0,"""803T>C""","""Ile268Thr""",1.0,1,3,11,12,null,null,null,null,527,null,3,6,4,8,11,"""H""","""MVK""","""MVK"""
"""2024-12-09_B11A1R1""","""G11""","""GFP""",154.027909,24.29425,140.0,155.0,168.0,171.0,178.0,183.0,194.0,178,896.607129,3527.121546,0.141705,0.180849,true,"""B11A1R1""","""B11A1R1-G11""","""B11A1R1_P1""","""G11""","""MVK""","""MVK_IIe268Thr""",null,null,null,"""1%_rescreen_P01_R1""","""D06""",44,"""G11""","""B11A1R1""","""B12A1R2""","""allele""",281.0,20.0,"""803T>C""","""Ile268Thr""",1.0,1,1,11,12,null,null,null,null,527,null,1,6,4,7,11,"""G""","""MVK""","""MVK"""
"""2024-12-09_B11A1R1""","""P01""","""GFP""",153.140249,19.709526,139.0,155.0,168.0,170.0,177.0,182.0,193.0,178,896.607129,3527.121546,0.142574,0.180849,true,"""B11A1R1""","""B11A1R1-P01""","""B11A1R1_P1""","""P01""","""MLH1""","""MLH1_Pro28Leu""","""VUSmut_GDEh03""","""G11""",87.0,"""1%_rescreen_P01_R1""","""H01""",8,"""P01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,449.0,"""83C>T""","""Pro28Leu""",1.0,1,3,11,12,null,null,null,null,527,null,3,1,8,16,1,"""P""","""MLH1""","""MLH1"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2024-12-09_B11A1R1""","""A24""","""GFP""",1413.407546,4593.042248,148.0,167.0,220.0,415.0,3317.0,7638.0,23704.0,178,896.607129,3527.121546,2.20456,0.180849,false,"""B11A1R1""","""B11A1R1-A24""","""B11A1R1_P1""","""A24""","""KLH3""","""KLH3""","""594 Controls """,null,null,"""1%_rescreen_P01_R1""","""A12""",89,"""A24""","""B11A1R1""","""B12A1R2""","""cPC""",null,null,null,null,null,1,2,11,12,null,null,null,null,594,null,2,12,1,1,24,"""A""","""KLH3""","""KLH3"""
"""2024-12-09_B11A1R1""","""A23""","""GFP""",1676.302456,4865.913679,150.0,169.0,343.0,954.0,4678.0,9514.0,24068.0,178,896.607129,3527.121546,2.205349,0.180849,false,"""B11A1R1""","""B11A1R1-A23""","""B11A1R1_P1"

gene_allele,well,is_bg_sum,Batch,QC_Flag
str,str,u32,str,str
"""GSS_Arg125Cys""","""O22""",0,"""2024_12_09_Batch_11""","""PASS"""
"""STXBP1_Leu256Pro""","""E17""",0,"""2024_12_09_Batch_11""","""PASS"""
"""MLH1_Ala681Val""","""D04""",0,"""2024_12_09_Batch_11""","""PASS"""
"""GSS_Arg125Cys""","""O21""",0,"""2024_12_09_Batch_11""","""PASS"""
"""MVK_Gly211Ala""","""N09""",0,"""2024_12_09_Batch_11""","""PASS"""
…,…,…,…,…
"""MLH1_Ala586Asp""","""B09""",0,"""2024_12_09_Batch_11""","""PASS"""
"""GSS_Arg125Cys""","""P21""",0,"""2024_12_09_Batch_11""","""PASS"""
"""MLH1_Ser295Thr""","""N08""",0,"""2024_12_09_Batch_11""","""PASS"""


plate,well,channel,mean,std,perc_25,perc_50,perc_75,perc_80,perc_90,perc_95,perc_99,median_plate,mean_plate,std_plate,s2n_ratio,s2n_threshold,is_bg,plate_id,plate_well,plate_map_name,well_position,symbol,gene_allele,source_plate,source_well,source_numb,dest_plate,dest_well,dest_numb,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,technical_replicate,batch1,batch2,clone_ID,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,vector,notes,quad,c96,r96,r384,c384,r384_letter,gene_symbol,symbol_original
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,bool,str,str,str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,f64,str,str,f64,i64,i64,i64,i64,str,str,str,str,i64,str,i64,i64,i64,i64,i64,str,str,str
"""2024-12-09_B11A1R1""","""H11""","""DAPI""",193.331611,141.487152,171.0,193.0,213.0,217.0,227.0,235.0,248.0,198,286.685087,382.698537,1.450292,1.652741,true,"""B11A1R1""","""B11A1R1-H11""","""B11A1R1_P1""","""H11""","""MVK""","""MVK_IIe268Thr""",null,null,null,"""1%_rescreen_P01_R1""","""D06""",44,"""H11""","""B11A1R1""","""B12A1R2""","""allele""",281.0,20.0,"""803T>C""","""Ile268Thr""",1.0,1,3,11,12,null,null,null,null,527,null,3,6,4,8,11,"""H""","""MVK""","""MVK"""
"""2024-12-09_B11A1R1""","""H12""","""DAPI""",194.885496,239.024922,172.0,194.0,214.0,219.0,229.0,237.0,253.0,198,286.685087,382.698537,1.47093,1.652741,true,"""B11A1R1""","""B11A1R1-H12""","""B11A1R1_P1""","""H12""","""MVK""","""MVK_IIe268Thr""",null,null,null,"""1%_rescreen_P01_R1""","""D06""",44,"""H12""","""B11A1R1""","""B12A1R2""","""allele""",281.0,20.0,"""803T>C""","""Ile268Thr""",1.0,1,4,11,12,null,null,null,null,527,null,4,6,4,8,12,"""H""","""MVK""","""MVK"""
"""2024-12-09_B11A1R1""","""G12""","""DAPI""",194.350682,231.000814,170.0,192.0,212.0,216.0,227.0,234.0,252.0,198,286.685087,382.698537,1.482353,1.652741,true,"""B11A1R1""","""B11A1R1-G12""","""B11A1R1_P1""","""G12""","""MVK""","""MVK_IIe268Thr""",null,null,null,"""1%_rescreen_P01_R1""","""D06""",44,"""G12""","""B11A1R1""","""B12A1R2""","""allele""",281.0,20.0,"""803T>C""","""Ile268Thr""",1.0,1,2,11,12,null,null,null,null,527,null,2,6,4,7,12,"""G""","""MVK""","""MVK"""
"""2024-12-09_B11A1R1""","""G11""","""DAPI""",196.679103,328.090804,169.0,192.0,212.0,217.0,227.0,236.0,274.0,198,286.685087,382.698537,1.621302,1.652741,true,"""B11A1R1""","""B11A1R1-G11""","""B11A1R1_P1""","""G11""","""MVK""","""MVK_IIe268Thr""",null,null,null,"""1%_rescreen_P01_R1""","""D06""",44,"""G11""","""B11A1R1""","""B12A1R2""","""allele""",281.0,20.0,"""803T>C""","""Ile268Thr""",1.0,1,1,11,12,null,null,null,null,527,null,1,6,4,7,11,"""G""","""MVK""","""MVK"""
"""2024-12-09_B11A1R1""","""P01""","""DAPI""",192.079061,122.294542,166.0,187.0,206.0,210.0,220.0,229.0,297.0,198,286.685087,382.698537,1.789157,1.652741,false,"""B11A1R1""","""B11A1R1-P01""","""B11A1R1_P1""","""P01""","""MLH1""","""MLH1_Pro28Leu""","""VUSmut_GDEh03""","""G11""",87.0,"""1%_rescreen_P01_R1""","""H01""",8,"""P01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,449.0,"""83C>T""","""Pro28Leu""",1.0,1,3,11,12,null,null,null,null,527,null,3,1,8,16,1,"""P""","""MLH1""","""MLH1"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2024-12-09_B12A1R2_widefield""","""J18""","""Mito""",2659.552717,4005.388884,1579.0,1811.0,2050.0,2167.0,4033.0,7388.0,16546.0,2613,3748.469075,3160.682644,10.478784,1.718722,false,"""widefield""","""widefield-J18""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2024-12-09_B12A1R2_widefield""","""E20""","""Mito""",2787.382706,3780.481204,1707.0,1938.0,2167.0,2244.0,4180.0,8008.0,18026.0,2613,3748.469075,3160.682644,10.560047,1.718722,false,"""widefield""","""widefield-E20""",null,null,null,null,null,null,null,null,null,nu

plate_map_name,well_position,symbol,gene_allele,source_plate,source_well,source_numb,dest_plate,dest_well,dest_numb,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,technical_replicate,batch1,batch2,clone_ID,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,vector,notes,quad,c96,r96,r384,c384,r384_letter,gene_symbol,symbol_original,plate_well
str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,f64,str,str,f64,i64,i64,i64,i64,str,str,str,str,i64,str,i64,i64,i64,i64,i64,str,str,str,str
"""B12A1R2_P1""","""A01""","""MLH1""","""MLH1""","""Tube""",null,null,"""1%_rescreen_P01_R1""","""A01""",1,"""A01""","""B11A1R1""","""B12A1R2""","""disease_wt""",2713.0,null,null,null,1.0,1,1,11,12,null,null,null,null,527,null,1,1,1,1,1,"""A""","""MLH1""","""MLH1""","""B12A1R2-A01"""
"""B12A1R2_P1""","""C01""","""MLH1""","""MLH1_Ala29Gly""","""VUSmut_GDEh03""","""H11""",88.0,"""1%_rescreen_P01_R1""","""B01""",2,"""C01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,451.0,"""86C>G""","""Ala29Gly""",1.0,1,1,11,12,null,null,null,null,527,null,1,1,2,3,1,"""C""","""MLH1""","""MLH1""","""B12A1R2-C01"""
"""B12A1R2_P1""","""E01""","""MLH1""","""MLH1_Asn38His""","""VUSmut_GDEh03""","""D12""",92.0,"""1%_rescreen_P01_R1""","""C01""",3,"""E01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,467.0,"""112A>C""","""Asn38His""",1.0,1,1,11,12,null,null,null,null,527,null,1,1,3,5,1,"""E""","""MLH1""","""MLH1""","""B12A1R2-E01"""
"""B12A1R2_P1""","""G01""","""MLH1""","""MLH1_Asn38Lys""","""VUSmut_GDEh03""","""E12""",93.0,"""1%_rescreen_P01_R1""","""D01""",4,"""G01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,470.0,"""114C>G""","""Asn38Lys""",1.0,1,1,11,12,null,null,null,null,527,null,1,1,4,7,1,"""G""","""MLH1""","""MLH1""","""B12A1R2-G01"""
"""B12A1R2_P1""","""I01""","""MLH1""","""MLH1_Cys39Tyr""","""VUSmut_GDEh03""","""F12""",94.0,"""1%_rescreen_P01_R1""","""E01""",5,"""I01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,472.0,"""116G>A""","""Cys39Tyr""",1.0,1,1,11,12,null,null,null,null,527,null,1,1,5,9,1,"""I""","""MLH1""","""MLH1""","""B12A1R2-I01"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""B12A1R2_P1""","""L22""","""AGXT""","""AGXT_Asp201Asn""","""594 Controls """,null,null,"""1%_rescreen_P01_R1""","""F11""",86,"""L22""","""B11A1R1""","""B12A1R2""","""cPC""",null,null,null,null,null,1,4,11,12,null,null,null,null,594,null,4,11,6,12,22,"""L""","""AGXT""","""AGXT""","""B12A1R2-L22"""
"""B12A1R2_P1""","""N22""","""GSS""","""GSS""","""594 Controls """,null,null,"""1%_rescreen_P01_R1""","""G11""",87,"""N22""","""B11A1R1""","""B12A1R2""","""cPC""",null,null,null,null,null,1,4,11,12,null,null,null,null,594,null,4,11,7,14,22,"""N""","""GSS""","""GSS""","""B12A1R2-N22"""
"""B12A1R2_P1""","""P22""","""GSS""","""GSS_Arg125Cys""","""594 Controls """,null,null,"""1%_rescreen_P01_R1""","""H11""",88,"""P22""","""B11A1R1""","""B12A1R2""","""cPC""",null,null,null,null,null,1,4,11,12,null,null,null,null,594,null,4,11,8,16,22,"""P""","""GSS""","""GSS""","""B12A1R2-P22"""


plate,well,channel,mean,std,perc_25,perc_50,perc_75,perc_80,perc_90,perc_95,perc_99,median_plate,mean_plate,std_plate,s2n_ratio,s2n_threshold,is_bg,plate_id,plate_well,plate_map_name,well_position,symbol,gene_allele,source_plate,source_well,source_numb,dest_plate,dest_well,dest_numb,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,…,plate_map_name_right,well_position_right,symbol_right,gene_allele_right,source_plate_right,source_well_right,source_numb_right,dest_plate_right,dest_well_right,dest_numb_right,imaging_well_right,imaging_plate_R1_right,imaging_plate_R2_right,node_type_right,orf_id_wt_right,ccsb_mutation_id_right,nt_change_cdna_right,aa_change_right,allele_set_right,imaging_plate_right,technical_replicate_right,batch1_right,batch2_right,clone_ID_right,hgmd_ID_right,clinvar_allele_ID_right,clinvar_clinical_significance_right,vector_right,notes_right,quad_right,c96_right,r96_right,r384_right,c384_right,r384_letter_right,gene_symbol_right,symbol_original_right
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,bool,str,str,str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,f64,str,…,str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,f64,str,str,f64,i64,i64,i64,i64,str,str,str,str,i64,str,i64,i64,i64,i64,i64,str,str,str
"""2024-12-09_B11A1R1""","""G12""","""GFP""",154.660478,33.916139,141.0,156.0,169.0,172.0,179.0,184.0,193.0,178,896.607129,3527.121546,0.13637,0.180849,true,"""B11A1R1""","""B11A1R1-G12""","""B11A1R1_P1""","""G12""","""MVK""","""MVK_IIe268Thr""",null,null,null,"""1%_rescreen_P01_R1""","""D06""",44,"""G12""","""B11A1R1""","""B12A1R2""","""allele""",281.0,20.0,"""803T>C""",…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2024-12-09_B11A1R1""","""H12""","""GFP""",155.102661,30.689846,141.0,156.0,169.0,172.0,179.0,184.0,193.0,178,896.607129,3527.121546,0.13637,0.180849,true,"""B11A1R1""","""B11A1R1-H12""","""B11A1R1_P1""","""H12""","""MVK""","""MVK_IIe268Thr""",null,null,null,"""1%_rescreen_P01_R1""","""D06""",44,"""H12""","""B11A1R1""","""B12A1R2""","""allele""",281.0,20.0,"""803T>C""",…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2024-12-09_B11A1R1""","""H11""","""GFP""",154.026189,19.641371,140.0,155.0,169.0,171.0,178.0,183.0,192.0,178,896.607129,3527.121546,0.137205,0.180849,true,"""B11A1R1""","""B11A1R1-H11""","""B11A1R1_P1""","""H11""","""MVK""","""MVK_IIe268Thr""",null,null,null,"""1%_rescreen_P01_R1""","""D06""",44,"""H11""","""B11A1R1""","""B12A1R2""","""allele""",281.0,20.0,"""803T>C""",…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2024-12-09_B11A1R1""","""G11""","""GFP""",154.027909,24.29425,140.0,155.0,168.0,171.0,178.0,183.0,194.0,178,896.607129,3527.121546,0.141705,0.180849,true,"""B11A1R1""","""B11A1R1-G11""","""B11A1R1_P1""","""G11""","""MVK""","""MVK_IIe268Thr""",null,null,null,"""1%_rescreen_P01_R1""","""D06""",44,"""G11""","""B11A1R1""","""B12A1R2""","""allele""",281.0,20.0,"""803T>C""",…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2024-12-09_B11A1R1""","""P01""","""GFP""",153.140249,19.709526,139.0,155.0,168.0,170.0,177.0,182.0,193.0,178,896.607129,3527.121546,0.142574,0.180849,true,"""B11A1R1""","""B11A1R1-P01""","""B11A1R1_P1""","""P01""","""MLH1""","""MLH1_Pro28Leu""","""VUSmut_GDEh03""","""G11""",87.0,"""1%_rescreen_P01_R1""","""H01""",8,"""P01""","""B11A1R1""","""B12A1R2""","""allele""",2713.0,449.0,"""83C>T""",…,null,null,null,null,null,n

gene_allele,well,is_bg_sum,Batch,QC_Flag
str,str,u32,str,str
"""MLH1_Ile36Met""","""M01""",0,"""2024_12_09_Batch_12""","""PASS"""
"""MLH1_Asn38His""","""F01""",0,"""2024_12_09_Batch_12""","""PASS"""
"""STXBP1_Arg235Gln""","""A13""",0,"""2024_12_09_Batch_12""","""PASS"""
"""STXBP1_Val451Ile""","""B21""",0,"""2024_12_09_Batch_12""","""PASS"""
"""STXBP1_Asp238Glu""","""G16""",0,"""2024_12_09_Batch_12""","""PASS"""
…,…,…,…,…
"""MLH1_Ala681Thr""","""A04""",0,"""2024_12_09_Batch_12""","""PASS"""
"""STXBP1_Leu256Pro""","""F18""",0,"""2024_12_09_Batch_12""","""PASS"""
"""MLH1_Pro747Ser""","""G06""",0,"""2024_12_09_Batch_12""","""PASS"""
